# Initial setup

In [87]:
import polars as pl
import polars.selectors as cs

from pathlib import Path
import os
import re

In [88]:
# Defining the folders that'll be used to load the files
# The processed files will be saved into a different folder and loaded in a separate notebook

files_raw = r'C:\Users\mpola\asthma analysis\files\parsed files'

In [89]:
# Getting all the non-html files in the raw files folder. The datasets are stored in .csv, .xls and .xlsx formats 
# but we also have a few .htm files that describe the schema of some of the datasets

files_raw_path = Path(files_raw)

csv_files = list(files_raw_path.glob('*.csv'))
xls_files = list(files_raw_path.glob('*.xls'))
xlsx_files = list(files_raw_path.glob('*.xlsx'))
txt_files = list(files_raw_path.glob('*.txt'))

In [90]:
xlsx_files

[WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/Medical doctors per state.xlsx'),
 WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/Respiratory therapists per state.xlsx')]

In [91]:
txt_files

[WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/counties gazette.txt')]

Loading the dataframes will be done one-by-one to ensure a naming convention that's easy to read and write out. The original filenames were difficult to humanly parse whereas the edited filenames are meant to be as verbose as possible. Working in Python, its better to maintain brevity, and storing the original name in a dictionary ensures we have a fallback to check what the dataset we're working on is meant to be.

In [92]:
# initiating the dictionary of dataframe name to file name connections and dataframes to store the datasets. 
filesdict = {}

df_raw = {}

In [93]:
# Writing out the list of names for the dataframes. Not all of these dataframes were ultimately used

csv_names_list = ['1yr_demographic',
                  '1yr_private_insurance',
                  '1yr_public_insurance',
                  '5yr_demographic',
                  '5yr_private_insurance',
                  '5yr_public_insurance',
                  'asthma_hospitalization',
                  'county_health_rankings',
                  'emergency_asthma',
                  'pollution',
                  'svi_2018',
                  'svi_2020']

xlsx_names_list = ['md_per_state',
                   'resp_therapist_per_state']

In [94]:
# Loading the csv files

for i, entry in enumerate(csv_files):
    filesdict[csv_names_list[i]] = entry
    df_raw[csv_names_list[i]] = pl.read_csv(entry)

In [95]:
# Loading the xlsx files

for i, entry in enumerate(xlsx_files):
    filesdict[xlsx_names_list[i]] = entry
    df_raw[xlsx_names_list[i]] = pl.read_excel(entry)

In [96]:
# Manually handling the xls file since there's only one but it has multiple tabs that we want to load
filesdict['county_health_measures'] = xls_files[0]
df_raw['county_health_measures'] = pl.read_excel(source=xls_files[0], 
                                                 sheet_name='Ranked Measure Data', 
                                                 #engine='openpyxl'
                                                )

filesdict['county_additional_measures'] = xls_files[0]
df_raw['county_additional_measures'] = pl.read_excel(source=xls_files[0], 
                                                     sheet_name='Additional Measure Data', 
                                                     #engine='openpyxl'
                                                    )

In [97]:
txt_files

[WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/counties gazette.txt')]

In [98]:
# The US Census Bureau's gazetteer file can be loaded with read_csv, but we need to specify the separator used to ensure the
# headers are registered without issue
df_raw['lat_long'] = pl.read_csv(source=txt_files[0])

df_raw['lat_long'] = (
    pl.read_csv(
        source = txt_files[0],
        separator = '\t',  # Census Gazetteer files are tab-separated
        has_header = True,
        # Forcing GEOID to be String to keep leading zeros (e.g., '01001')
        schema_overrides={
            'GEOID': pl.String,
            'ANSICODE': pl.String, 
            'USPS': pl.String
        },
        # Handling the messy trailing whitespace often found in these files
        truncate_ragged_lines=True,
        ignore_errors=True
    )
)

df_raw['lat_long']

USPS,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
str,str,str,str,i64,i64,f64,f64,f64,str
"""AL""","""01001""","""00161526""","""Autauga County""",1539634184,25674812,594.456,9.913,32.532237,"""-86.64644 …"
"""AL""","""01003""","""00161527""","""Baldwin County""",4117656514,1132955729,1589.836,437.437,30.659218,"""-87.746067 …"
"""AL""","""01005""","""00161528""","""Barbour County""",2292160149,50523213,885.008,19.507,31.870253,"""-85.405104 …"
"""AL""","""01007""","""00161529""","""Bibb County""",1612188717,9572303,622.47,3.696,33.015893,"""-87.127148 …"
"""AL""","""01009""","""00161530""","""Blount County""",1670259090,14860281,644.891,5.738,33.977358,"""-86.56644 …"
…,…,…,…,…,…,…,…,…,…
"""PR""","""72145""","""01804553""","""Vega Baja Municipio""",118746309,57826361,45.848,22.327,18.455128,"""-66.397883 …"
"""PR""","""72147""","""01804554""","""Vieques Municipio""",131541389,552192821,50.788,213.203,18.125418,"""-65.432474 …"
"""PR""","""72149""","""01804555""","""Villalba Municipio""",92298569,3622640,35.637,1.399,18.130718,"""-66.472244 …"


# Data cleaning and reformatting

In [99]:
filesdict

{'1yr_demographic': WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/1 Year estimate population by age, gender and dependency per county (ACSST1Y2019.S0101).csv'),
 '1yr_private_insurance': WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/1 Year estimate private health insurance coverage (ACSST1Y2019.S2703).csv'),
 '1yr_public_insurance': WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/1 Year estimate public health insurance coverage (ACSST1Y2019.S2704).csv'),
 '5yr_demographic': WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/5 Year estimate population by age, gender and dependency per county (ACSST5Y2019.S0101).csv'),
 '5yr_private_insurance': WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/5 Year estimate private health insurance coverage (ACSST5Y2019.S2703).csv'),
 '5yr_public_insurance': WindowsPath('C:/Users/mpola/asthma analysis/files/parsed files/5 Year estimate public health insurance coverage (ACSST5Y201

In [100]:
df_clean = {}

## Latitude/Longitude

In [101]:
# For this dataframe we're mostly interested in the latitude and longitude information
df_clean['lat_long'] = (df_raw['lat_long'].with_columns(
        pl.col(pl.String).str.strip_chars()
    )
    .rename({
        'USPS': 'State Abbreviation',
        #'GEOID': 'FIPS',
        'NAME': 'County',
        'ALAND': 'Land Area',
        'AWATER': 'Water Area',
        #'ALAND_SQMI': 'Land Area (Square Miles)',
        #'AWATER_SQMI': 'Water Area (Square Miles)',
        'INTPTLAT': 'Latitude',
        'INTPTLONG                                                                                                               ': 'Longitude' # The excessive spaces are an artifact of tsv loading and are, unfortunately, necessary
    })
    .drop([
        'ANSICODE',
        'GEOID',
        'ALAND_SQMI',
        'AWATER_SQMI'
    ])
)

df_clean['lat_long']

State Abbreviation,County,Land Area,Water Area,Latitude,Longitude
str,str,i64,i64,f64,str
"""AL""","""Autauga County""",1539634184,25674812,32.532237,"""-86.64644"""
"""AL""","""Baldwin County""",4117656514,1132955729,30.659218,"""-87.746067"""
"""AL""","""Barbour County""",2292160149,50523213,31.870253,"""-85.405104"""
"""AL""","""Bibb County""",1612188717,9572303,33.015893,"""-87.127148"""
"""AL""","""Blount County""",1670259090,14860281,33.977358,"""-86.56644"""
…,…,…,…,…,…
"""PR""","""Vega Baja Municipio""",118746309,57826361,18.455128,"""-66.397883"""
"""PR""","""Vieques Municipio""",131541389,552192821,18.125418,"""-65.432474"""
"""PR""","""Villalba Municipio""",92298569,3622640,18.130718,"""-66.472244"""


## 5 Year estimate population by age, gender and dependency per county

(The following information is contained in the table notes text file bundled with the dataset, and is presented with only formatting changes to ensure the information renders correctly in the Markdown environment)


Although the American Community Survey (ACS) produces population, demographic and housing unit estimates, it is the Census Bureau's Population Estimates Program that produces and disseminates the official estimates of the population for the nation, states, counties, cities, and towns and estimates of housing units for states and counties.<br/><br/>

-------------------------------------------------------------------------------------------------------------
Supporting documentation on code lists, subject definitions, data accuracy, and statistical testing can be found on the American Community Survey website in the <a href='https://www.census.gov/programs-surveys/acs/technical-documentation/code-lists.html'>Technical Documentation</a> section.<br /><br />Sample size and data quality measures (including coverage rates, allocation rates, and response rates) can be found on the American Community Survey website in the <a href='https://www.census.gov/acs/www/methodology/sample_size_and_data_quality/'>Methodology</a> section.<br/><br/>

-------------------------------------------------------------------------------------------------------------
Source: U.S. Census Bureau, 2019 American Community Survey 5-Year Estimates<br/><br/>

-------------------------------------------------------------------------------------------------------------
Data are based on a sample and are subject to sampling variability. The degree of uncertainty for an estimate arising from sampling variability is represented through the use of a margin of error. The value shown here is the 90 percent margin of error. The margin of error can be interpreted roughly as providing a 90 percent probability that the interval defined by the estimate minus the margin of error and the estimate plus the margin of error (the lower and upper confidence bounds) contains the true value. In addition to sampling variability, the ACS estimates are subject to nonsampling error (for a discussion of nonsampling variability, see <a href='https://www.census.gov/programs-surveys/acs/technical-documentation.html'>ACS Technical Documentation</a>). The effect of nonsampling error is not represented in these tables.<br/><br/>

-------------------------------------------------------------------------------------------------------------

The age dependency ratio is derived by dividing the combined under-18 and 65-and-over populations by the 18-to-64 population and multiplying by 100.<br/><br/>

-------------------------------------------------------------------------------------------------------------

The old-age dependency ratio is derived by dividing the population 65 and over by the 18-to-64 population and multiplying by 100.<br/><br/>

-------------------------------------------------------------------------------------------------------------

The child dependency ratio is derived by dividing the population under 18 by the 18-to-64 population and multiplying by 100.<br/><br/>

-------------------------------------------------------------------------------------------------------------

When information is missing or inconsistent, the Census Bureau logically assigns an acceptable value using the response to a related question or questions. If a logical assignment is not possible, data are filled using a statistical process called allocation, which uses a similar individual or household to provide a donor value. The 'Allocated' section is the number of respondents who received an allocated value for a particular subject.<br/><br/>

-------------------------------------------------------------------------------------------------------------

The 2019 American Community Survey (ACS) data generally reflect the September 2018 Office of Management and Budget (OMB) delineations of metropolitan and micropolitan statistical areas. In certain instances the names, codes, and boundaries of the principal cities shown in ACS tables may differ from the OMB delineations due to differences in the effective dates of the geographic entities.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Estimates of urban and rural populations, housing units, and characteristics reflect boundaries of urban areas defined based on Census 2010 data. As a result, data for urban and rural areas from the ACS do not necessarily reflect the results of ongoing urbanization.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Explanation of Symbols:<ol><li>An '\*\*' entry in the margin of error column indicates that either no sample observations or too few sample observations were available to compute a standard error and thus the margin of error. A statistical test is not appropriate.</li><li>An '-' entry in the estimate column indicates that either no sample observations or too few sample observations were available to compute an estimate, or a ratio of medians cannot be calculated because one or both of the median estimates falls in the lowest interval or upper interval of an open-ended distribution, or the margin of error associated with a median was larger than the median itself.</li><li>An '-' following a median estimate means the median falls in the lowest interval of an open-ended distribution.</li><li>An '+' following a median estimate means the median falls in the upper interval of an open-ended distribution.</li><li>An '\*\*\*' entry in the margin of error column indicates that the median falls in the lowest interval or upper interval of an open-ended distribution. A statistical test is not appropriate.</li><li>An '\*\*\*\*\*' entry in the margin of error column indicates that the estimate is controlled. A statistical test for sampling variability is not appropriate. </li><li>An 'N' entry in the estimate and margin of error columns indicates that data for this geographic area cannot be displayed because the number of sample cases is too small.</li><li>An '(X)' means that the estimate is not applicable or not available.</li></ol><br/><br/>

-------------------------------------------------------------------------------------------------------------

In [102]:
# When loading the dataframe, the first row gets loaded as a regular row whereas it should be the 
# headers instead. The existing headers use a code system that's not directly understandable
df_raw['5yr_demographic']

GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C01_002E,S0101_C01_002M,S0101_C01_003E,S0101_C01_003M,S0101_C01_004E,S0101_C01_004M,S0101_C01_005E,S0101_C01_005M,S0101_C01_006E,S0101_C01_006M,S0101_C01_007E,S0101_C01_007M,S0101_C01_008E,S0101_C01_008M,S0101_C01_009E,S0101_C01_009M,S0101_C01_010E,S0101_C01_010M,S0101_C01_011E,S0101_C01_011M,S0101_C01_012E,S0101_C01_012M,S0101_C01_013E,S0101_C01_013M,S0101_C01_014E,S0101_C01_014M,S0101_C01_015E,S0101_C01_015M,S0101_C01_016E,S0101_C01_016M,S0101_C01_017E,S0101_C01_017M,S0101_C01_018E,…,S0101_C06_021E,S0101_C06_021M,S0101_C06_022E,S0101_C06_022M,S0101_C06_023E,S0101_C06_023M,S0101_C06_024E,S0101_C06_024M,S0101_C06_025E,S0101_C06_025M,S0101_C06_026E,S0101_C06_026M,S0101_C06_027E,S0101_C06_027M,S0101_C06_028E,S0101_C06_028M,S0101_C06_029E,S0101_C06_029M,S0101_C06_030E,S0101_C06_030M,S0101_C06_031E,S0101_C06_031M,S0101_C06_032E,S0101_C06_032M,S0101_C06_033E,S0101_C06_033M,S0101_C06_034E,S0101_C06_034M,S0101_C06_035E,S0101_C06_035M,S0101_C06_036E,S0101_C06_036M,S0101_C06_037E,S0101_C06_037M,S0101_C06_038E,S0101_C06_038M,
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Geography""","""Geographic Area Name""","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…","""Margin of Error!!Total!!Total …","""Estimate!!Total!!Total populat…",…,"""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…","""Estimate!!Percent Female!!Tota…","""Margin of Error!!Percent Femal…",null
"""0500000US0100

In [103]:
# Extracting the first row as a tuple, converting to list, and cleaning it
descriptions = df_raw['5yr_demographic'].row(0)
descriptions

('Geography',
 'Geographic Area Name',
 'Estimate!!Total!!Total population',
 'Margin of Error!!Total!!Total population',
 'Estimate!!Total!!Total population!!AGE!!Under 5 years',
 'Margin of Error!!Total!!Total population!!AGE!!Under 5 years',
 'Estimate!!Total!!Total population!!AGE!!5 to 9 years',
 'Margin of Error!!Total!!Total population!!AGE!!5 to 9 years',
 'Estimate!!Total!!Total population!!AGE!!10 to 14 years',
 'Margin of Error!!Total!!Total population!!AGE!!10 to 14 years',
 'Estimate!!Total!!Total population!!AGE!!15 to 19 years',
 'Margin of Error!!Total!!Total population!!AGE!!15 to 19 years',
 'Estimate!!Total!!Total population!!AGE!!20 to 24 years',
 'Margin of Error!!Total!!Total population!!AGE!!20 to 24 years',
 'Estimate!!Total!!Total population!!AGE!!25 to 29 years',
 'Margin of Error!!Total!!Total population!!AGE!!25 to 29 years',
 'Estimate!!Total!!Total population!!AGE!!30 to 34 years',
 'Margin of Error!!Total!!Total population!!AGE!!30 to 34 years',
 'Estimat

In [104]:
# A lot of these columnns are redundant for our work, so we can filter out most of them to help readability and memory usage
# For brevity, we keep only the columns containing percent estimates since population density and total population varies
# wildly from county to county
# We fetch the 'SUMMARY INDICATORS' columns since they contain information that may be relevant.
# We also fetch the under-18 percentage to compare asthma in adolescence

cols_to_keep = []

# We zip the current column ID (header) with the description (row 0)
for col_id, desc in zip(df_raw['5yr_demographic'].columns, descriptions):
    
    # Handling potential None values if there are empty columns, just in case
    if desc is None:
        continue
        
    # We typically also want to keep 'Geography' or 'id' for merging later
    is_percentage = 'Estimate!!Percent' in desc

    is_population = desc == 'Estimate!!Total!!Total population' 
    
    is_id = desc in ['Geographic Area Name', 'Geography']
    
    is_summary = 'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!' in desc
        
    if is_percentage or is_population or is_id or is_summary:
        # Selecting the column by its ID, but renaming it to the description
        cols_to_keep.append(pl.col(col_id).alias(desc))

# Performing the selection
df_clean['5yr_demographic'] = df_raw['5yr_demographic'].select(cols_to_keep).slice(1)

In [105]:
rename_map_demographic = {}
prefix_to_remove = 'Estimate!!'

for col in df_clean['5yr_demographic'].columns:
    if col.startswith(prefix_to_remove):
        
        new_name = col.replace(prefix_to_remove, '', 1)
        
        new_name = new_name.replace('!!', ' => ')
        
        rename_map_demographic[col] = new_name

# Applying the renaming
df_clean['5yr_demographic'] = df_clean['5yr_demographic'].rename(rename_map_demographic)
df_clean['5yr_demographic']

Geography,Geographic Area Name,Total => Total population,Total => Total population => SUMMARY INDICATORS => Median age (years),Total => Total population => SUMMARY INDICATORS => Sex ratio (males per 100 females),Total => Total population => SUMMARY INDICATORS => Age dependency ratio,Total => Total population => SUMMARY INDICATORS => Old-age dependency ratio,Total => Total population => SUMMARY INDICATORS => Child dependency ratio,Percent => Total population,Percent => Total population => AGE => Under 5 years,Percent => Total population => AGE => 5 to 9 years,Percent => Total population => AGE => 10 to 14 years,Percent => Total population => AGE => 15 to 19 years,Percent => Total population => AGE => 20 to 24 years,Percent => Total population => AGE => 25 to 29 years,Percent => Total population => AGE => 30 to 34 years,Percent => Total population => AGE => 35 to 39 years,Percent => Total population => AGE => 40 to 44 years,Percent => Total population => AGE => 45 to 49 years,Percent => Total population => AGE => 50 to 54 years,Percent => Total population => AGE => 55 to 59 years,Percent => Total population => AGE => 60 to 64 years,Percent => Total population => AGE => 65 to 69 years,Percent => Total population => AGE => 70 to 74 years,Percent => Total population => AGE => 75 to 79 years,Percent => Total population => AGE => 80 to 84 years,Percent => Total population => AGE => 85 years and over,Percent => Total population => SELECTED AGE CATEGORIES => 5 to 14 years,Percent => Total population => SELECTED AGE CATEGORIES => 15 to 17 years,Percent => Total population => SELECTED AGE CATEGORIES => Under 18 years,Percent => Total population => SELECTED AGE CATEGORIES => 18 to 24 years,Percent => Total population => SELECTED AGE CATEGORIES => 15 to 44 years,Percent => Total population => SELECTED AGE CATEGORIES => 16 years and over,Percent => Total population => SELECTED AGE CATEGORIES => 18 years and over,Percent => Total population => SELECTED AGE CATEGORIES => 21 years and over,Percent => Total population => SELECTED AGE CATEGORIES => 60 years and over,Percent => Total population => SELECTED AGE CATEGORIES => 62 years and over,…,Percent Female => Total population => AGE => Under 5 years,Percent Female => Total population => AGE => 5 to 9 years,Percent Female => Total population => AGE => 10 to 14 years,Percent Female => Total population => AGE => 15 to 19 years,Percent Female => Total population => AGE => 20 to 24 years,Percent Female => Total population => AGE => 25 to 29 years,Percent Female => Total population => AGE => 30 to 34 years,Percent Female => Total population => AGE => 35 to 39 years,Percent Female => Total population => AGE => 40 to 44 years,Percent Female => Total population => AGE => 45 to 49 years,Percent Female => Total population => AGE => 50 to 54 years,Percent Female => Total population => AGE => 55 to 59 years,Percent Female => Total population => AGE => 60 to 64 years,Percent Female => Total population => AGE => 65 to 69 years,Percent Female => Total population => AGE => 70 to 74 years,Percent Female => Total population => AGE => 75 to 79 years,Percent Female => Total population => AGE => 80 to 84 years,Percent Female => Total population => AGE => 85 years and over,Percent Female => Total population => SELECTED AGE CATEGORIES => 5 to 14 years,Percent Female => Total population => SELECTED AGE CATEGORIES => 15 to 17 years,Percent Female => Total population => SELECTED AGE CATEGORIES => Under 18 years,Percent Female => Total population => SELECTED AGE CATEGORIES => 18 to 24 years,Percent Female => Total population => SELECTED AGE CATEGORIES => 15 to 44 years,Percent Female => Total population => SELECTED AGE CATEGORIES => 16 years and over,Percent Female => Total population => SELECTED AGE CATEGORIES => 18 years and over,Percent Female => Total population => SELECTED AGE CATEGORIES => 21 years and over,Percent Female => Total population => SELECTED AGE CATEGORIES => 60 years and over,Percent Female => Total pop

In [106]:
# Separating the 'Geographic Area Name' column into 'County' and 'State' columns

def split_geo_area_name(df):
    """
    Takes a dataframe's 'Geographic Area Name' column and splits it into 'County' and 'State' columns
    """

    df_split = df.with_columns(
        # Getting the part before the comma (index 0)
        pl.col('Geographic Area Name')
          .str.split(', ')
          .list.get(0)
          .alias('County'),
        
        # Getting the part after the comma (index 1)
        pl.col('Geographic Area Name')
          .str.split(', ')
          .list.get(1)
          .alias('State')
    )
    
    # Reordering the columns to pull the county and state info to the front
    df_split = df_split.select(
    'County',
    'State',
    pl.exclude(['County', 'State'])
    )

        
    return df_split

# Dropping the original column
df_clean['5yr_demographic'] = split_geo_area_name(df_clean['5yr_demographic'])

In [107]:
# Defining a function to convert string columns to float
# A lot of the columns that store numeric data have exceptions that require them
# to be stored as strings instead. We will simply exclude these problematic rows
# when running the analysis, so we can replace them with null entries.

def convert_to_float(df, str_cols):
    """
    Converts the columns of a dataframe into float, excluding the columns in a given list.
    """

    df_converted = df.with_columns(
    pl.exclude(str_cols)                # Selecting all columns EXCEPT the text ones
      .str.replace_all(',', '')         # Removing commas (e.g. '1,500' -> '1500') to avoid formatting-related issues with potential large numbers
      .cast(pl.Float64, strict=False)   # Converting to float; turns non-numbers like '(X)' into null since these entries have specific meanings we can't blanket fill them
)
    return df_converted

In [108]:
text_cols = ['County', 'State', 'Geography']

df_clean['5yr_demographic'] = convert_to_float(df_clean['5yr_demographic'], text_cols)

## 5 Year estimate public and private health insurance coverage

(The following information is contained in the table notes text file bundled with the dataset, and is presented with only formatting changes to ensure the information renders correctly in the Markdown environment. The notes are the same for the public and the private health insurance coverage datasets)

Although the American Community Survey (ACS) produces population, demographic and housing unit estimates, it is the Census Bureau's Population Estimates Program that produces and disseminates the official estimates of the population for the nation, states, counties, cities, and towns and estimates of housing units for states and counties.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Supporting documentation on code lists, subject definitions, data accuracy, and statistical testing can be found on the American Community Survey website in the <a href='https://www.census.gov/programs-surveys/acs/technical-documentation/code-lists.html'>Technical Documentation</a> section.<br/><br/>Sample size and data quality measures (including coverage rates, allocation rates, and response rates) can be found on the American Community Survey website in the <a href='https://www.census.gov/acs/www/methodology/sample_size_and_data_quality/'>Methodology</a> section.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Source: U.S. Census Bureau, 2019 American Community Survey 5-Year Estimates<br/><br/>

-------------------------------------------------------------------------------------------------------------

Data are based on a sample and are subject to sampling variability. The degree of uncertainty for an estimate arising from sampling variability is represented through the use of a margin of error. The value shown here is the 90 percent margin of error. The margin of error can be interpreted roughly as providing a 90 percent probability that the interval defined by the estimate minus the margin of error and the estimate plus the margin of error (the lower and upper confidence bounds) contains the true value. In addition to sampling variability, the ACS estimates are subject to nonsampling error (for a discussion of nonsampling variability, see <a href='https://www.census.gov/programs-surveys/acs/technical-documentation.html'>ACS Technical Documentation</a>). The effect of nonsampling error is not represented in these tables.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Logical coverage edits applying a rules-based assignment of Medicaid, Medicare and military health coverage were added as of 2009 -- please see <a href='https://www.census.gov/library/working-papers/2010/demo/coverage_edits_final.html'>https://www.census.gov/library/working-papers/2010/demo/coverage_edits_final.html</a> for more details. Select geographies of 2008 data comparable to the 2009 and later tables are available at <a href='https://www.census.gov/data/tables/time-series/acs/1-year-re-run-health-insurance.html'>https://www.census.gov/data/tables/time-series/acs/1-year-re-run-health-insurance.html</a>. The health insurance coverage category names were modified in 2010. See <a href='https://www.census.gov/topics/health/health-insurance/about/glossary.html#par_textimage_18'>https://www.census.gov/topics/health/health-insurance/about/glossary.html#par_textimage_18</a> for a list of the insurance type definitions.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Beginning in 2017, selected variable categories were updated, including age-categories, income-to-poverty ratio (IPR) categories, and the age universe for certain employment and education variables. See user note entitled <a href='https://www.census.gov/programs-surveys/acs/technical-documentation/user-notes.html'>'Health Insurance Table Updates'</a> for further details.<br/><br/>

-------------------------------------------------------------------------------------------------------------

The 2019 American Community Survey (ACS) data generally reflect the September 2018 Office of Management and Budget (OMB) delineations of metropolitan and micropolitan statistical areas. In certain instances the names, codes, and boundaries of the principal cities shown in ACS tables may differ from the OMB delineations due to differences in the effective dates of the geographic entities.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Estimates of urban and rural populations, housing units, and characteristics reflect boundaries of urban areas defined based on Census 2010 data. As a result, data for urban and rural areas from the ACS do not necessarily reflect the results of ongoing urbanization.<br/><br/>

-------------------------------------------------------------------------------------------------------------

Explanation of Symbols:<ol><li>An \'\*\*\' entry in the margin of error column indicates that either no sample observations or too few sample observations were available to compute a standard error and thus the margin of error. A statistical test is not appropriate.</li><li>An \'-\' entry in the estimate column indicates that either no sample observations or too few sample observations were available to compute an estimate, or a ratio of medians cannot be calculated because one or both of the median estimates falls in the lowest interval or upper interval of an open-ended distribution, or the margin of error associated with a median was larger than the median itself.</li><li>An \'-\' following a median estimate means the median falls in the lowest interval of an open-ended distribution.</li><li>An \'+\' following a median estimate means the median falls in the upper interval of an open-ended distribution.</li><li>An \'***\' entry in the margin of error column indicates that the median falls in the lowest interval or upper interval of an open-ended distribution. A statistical test is not appropriate.</li><li>An \'\*\*\*\**\' entry in the margin of error column indicates that the estimate is controlled. A statistical test for sampling variability is not appropriate. </li><li>An \'N\' entry in the estimate and margin of error columns indicates that data for this geographic area cannot be displayed because the number of sample cases is too small.</li><li>An \'(X)\' means that the estimate is not applicable or not available.</li></ol><br/><br/>

-------------------------------------------------------------------------------------------------------------


In [109]:
descriptions_private = df_raw['5yr_private_insurance'].row(0)
descriptions_private

('Geography',
 'Geographic Area Name',
 'Estimate!!Total!!Civilian noninstitutionalized population',
 'Margin of Error!!Total!!Civilian noninstitutionalized population',
 'Estimate!!Total!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination',
 'Margin of Error!!Total!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination',
 'Estimate!!Total!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination!!Under 19',
 'Margin of Error!!Total!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination!!Under 19',
 'Estimate!!Total!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination!!19 to 64 years',
 'Margin of Err

In [110]:
descriptions_public = df_raw['5yr_public_insurance'].row(0)
descriptions_public

('Geography',
 'Geographic Area Name',
 'Estimate!!Total!!Civilian noninstitutionalized population',
 'Margin of Error!!Total!!Civilian noninstitutionalized population',
 'Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination',
 'Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination',
 'Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19',
 'Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19',
 'Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years',
 'Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years',
 'Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over',
 'Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!

In [111]:
# This ACS dataframe is very similar to the one before in terms of formatting and schema, so a lot of the steps we need to take are very similar
# with the exception of which columns to keep and how to handle them.

cols_to_keep = []

# We zip the current column ID (header) with the description (row 0)
for col_id, desc in zip(df_raw['5yr_private_insurance'].columns, descriptions_private):
    
    # Handling potential None values if there are empty columns, just in case
    if desc is None:
        continue
        
    # We typically also want to keep 'Geography' or 'id' for merging later
    is_id = desc in ['Geographic Area Name', 'Geography']

    # Here rather than following a specific logic we define the subset of columns that are relevant to us (coverage by cohort) to sort out later
    is_percent = 'Estimate!!Percent' in desc 
    
    if is_percent or is_id:
        # Selecting the column by its ID, but rename (alias) it to the description
        cols_to_keep.append(pl.col(col_id).alias(desc))

# Performing the selection
df_clean['5yr_private_insurance'] = df_raw['5yr_private_insurance'].select(cols_to_keep).slice(1)

#######################

# And repeating the same step but for the public insurance dataframe

cols_to_keep = []

for col_id, desc in zip(df_raw['5yr_public_insurance'].columns, descriptions_public):
    
    if desc is None:
        continue
        
    is_id = desc in ['Geographic Area Name', 'Geography']

    is_percent = 'Estimate!!Percent' in desc 
    
    if is_percent or is_id:
        cols_to_keep.append(pl.col(col_id).alias(desc))

# Performing the selection
df_clean['5yr_public_insurance'] = df_raw['5yr_public_insurance'].select(cols_to_keep).slice(1)

In [112]:
df_clean['5yr_private_insurance'].columns

['Geography',
 'Geographic Area Name',
 'Estimate!!Percent Private Coverage!!Civilian noninstitutionalized population',
 'Estimate!!Percent Private Coverage!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination',
 'Estimate!!Percent Private Coverage!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination!!Under 19',
 'Estimate!!Percent Private Coverage!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination!!19 to 64 years',
 'Estimate!!Percent Private Coverage!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Employer-based health insurance alone or in combination!!65 years and over',
 'Estimate!!Percent Private Coverage!!Civilian noninstitutionalized population!!COVERAGE ALONE OR IN COMBINATION!!Direct-purchase health insurance 

In [113]:
df_clean['5yr_public_insurance'].columns

['Geography',
 'Geographic Area Name',
 'Estimate!!Percent Public Coverage!!Civilian noninstitutionalized population',
 'Estimate!!Percent Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination',
 'Estimate!!Percent Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19',
 'Estimate!!Percent Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years',
 'Estimate!!Percent Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over',
 'Estimate!!Percent Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination',
 'Estimate!!Percent Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19',
 'Estimate!!Percent Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested pub

In [114]:
# Rather than trying to brute force appropriate names for the columns here, it's easier to just erase the redundant prefixes and reformat
# the !! separations into something easier to read

prefix_to_remove = 'Estimate!!'

rename_map_private = {}

for col in df_clean['5yr_private_insurance'].columns:
    # Checking if the column starts with the specific prefix, to avoid messing up the geography name
    if col.startswith(prefix_to_remove):
        
        # Removing the prefix
        new_name = col.replace(prefix_to_remove, '', 1)
        
        # Replacing the remaining '!!' with ' => '
        new_name = new_name.replace('!!', ' => ')
        
        # Adding to the dictionary
        rename_map_private[col] = new_name

# Applying the renaming
df_clean['5yr_private_insurance'] = df_clean['5yr_private_insurance'].rename(rename_map_private)

# Checking the results
print(df_clean['5yr_private_insurance'].columns[:5])

############################################################################

rename_map_public = {}

for col in df_clean['5yr_public_insurance'].columns:
    if col.startswith(prefix_to_remove):
        
        new_name = col.replace(prefix_to_remove, '', 1)
        
        new_name = new_name.replace('!!', ' => ')
        
        rename_map_public[col] = new_name

# Applying the renaming
df_clean['5yr_public_insurance'] = df_clean['5yr_public_insurance'].rename(rename_map_public)

# Checking the results
print(df_clean['5yr_public_insurance'].columns[:5])

['Geography', 'Geographic Area Name', 'Percent Private Coverage => Civilian noninstitutionalized population', 'Percent Private Coverage => Civilian noninstitutionalized population => COVERAGE ALONE OR IN COMBINATION => Employer-based health insurance alone or in combination', 'Percent Private Coverage => Civilian noninstitutionalized population => COVERAGE ALONE OR IN COMBINATION => Employer-based health insurance alone or in combination => Under 19']
['Geography', 'Geographic Area Name', 'Percent Public Coverage => Civilian noninstitutionalized population', 'Percent Public Coverage => COVERAGE ALONE OR IN COMBINATION => Medicare coverage alone or in combination', 'Percent Public Coverage => COVERAGE ALONE OR IN COMBINATION => Medicare coverage alone or in combination => Under 19']


In [115]:
# Utilizing the function from before to split county and state names into two columns
df_clean['5yr_private_insurance'] = split_geo_area_name(df_clean['5yr_private_insurance'])

df_clean['5yr_public_insurance'] = split_geo_area_name(df_clean['5yr_public_insurance'])

In [116]:
# Converting the columns into floats

text_cols = ['County', 'State', 'Geography']

df_clean['5yr_private_insurance'] = convert_to_float(df_clean['5yr_private_insurance'], text_cols)

df_clean['5yr_public_insurance'] = convert_to_float(df_clean['5yr_public_insurance'], text_cols)


In [117]:
# These dataframes have the word 'health' in the columns, which is not only redundant but also actively harmful
# because they can get mixed up with other columns from other dataframes that store health-related features

def remove_words_from_headers(df, words_to_remove):
    """
    Renames columns by removing a list of specific substrings from the headers.
    """
    rename_map = {}
    
    for col in df.columns:
        new_name = col
        # Iterating through every word in the list and removing it from the current column name
        for word in words_to_remove:
            new_name = new_name.replace(word, '')
        
        rename_map[col] = new_name
    
    return df.rename(rename_map)

In [118]:
# Running the function
# The space left after each word is deliberate here, since these words are always before 'insurance' or
# 'care' and doing so ensures that there are no double spaces after removing them.

words_to_remove = ['Health ', 'HEALTH ', 'health ']

df_clean['5yr_private_insurance'] = remove_words_from_headers(df_clean['5yr_private_insurance'], words_to_remove)
df_clean['5yr_public_insurance'] = remove_words_from_headers(df_clean['5yr_public_insurance'], words_to_remove)

## Asthma and air pollutant data per county

These datasets are much more straightforward and easy to work with than ACS datasets, so they're handled at once.

In [119]:
# Renaming the 'Value' columns to better represent what they describe. 

df_clean['asthma_hospitalization'] = df_raw['asthma_hospitalization'].rename(
    {'Value': 'Age-adjusted Hospitalization Rate for Asthma per 10,000 People',
    'Data Comment': 'Hospitalization Comment',
    'CountyFIPS': 'FIPS'})

df_clean['emergency_asthma'] = df_raw['emergency_asthma'].rename(
    {'Value': 'Age-adjusted ER Visit Rate for Asthma per 10,000 People',
    'Data Comment': 'ER Visit Comment',
    'CountyFIPS': 'FIPS'})

In [120]:
# Dropping the redundant columns and reordering the remainder.

df_clean['asthma_hospitalization'] = df_clean['asthma_hospitalization'].select(
'County',
'State',
pl.exclude(['County', 'State']).exclude('StateFIPS', 'Year', '')
)

df_clean['emergency_asthma'] = df_clean['emergency_asthma'].select(
'County',
'State',
pl.exclude(['County', 'State']).exclude('StateFIPS', 'Year', '')
)

In [121]:
df_clean['emergency_asthma']

County,State,FIPS,"Age-adjusted ER Visit Rate for Asthma per 10,000 People",ER Visit Comment
str,str,i64,str,str
"""Apache""","""Arizona""",4001,"""30.5""",""""""
"""Cochise""","""Arizona""",4003,"""38.8""",""""""
"""Coconino""","""Arizona""",4005,"""19.7""",""""""
"""Gila""","""Arizona""",4007,"""46.2""",""""""
"""Graham""","""Arizona""",4009,"""31.9""",""""""
…,…,…,…,…
"""Waukesha""","""Wisconsin""",55133,"""19.8""",""""""
"""Waupaca""","""Wisconsin""",55135,"""39.7""",""""""
"""Waushara""","""Wisconsin""",55137,"""28.5""",""""""


In [122]:
# Converting the asthma dataframe's columns into floats

text_cols = ['State', 'County', 'Hospitalization Comment', 'FIPS']

df_clean['asthma_hospitalization'] = convert_to_float(df_clean['asthma_hospitalization'], text_cols)

######################

text_cols = ['State', 'County', 'ER Visit Comment', 'FIPS']

df_clean['emergency_asthma'] = convert_to_float(df_clean['emergency_asthma'], text_cols)


In [123]:
# We will need to pivot the pollution dataframe since each pollutant has its own row, 
# but we want them to have their own columns

# Limiting the list to the year 2019
df_clean['pollution'] = (df_raw['pollution']
                         .filter(pl.col('Year')==2019)
                         .rename({'CountyFIPS': 'FIPS'})
)

df_clean['pollution'] = df_clean['pollution'].pivot(
    values=['Value', 'Data Comment'],          
    index=['County', 'State', 'FIPS'],  
    on='Pollutant'           
)

# Checking the result
df_clean['pollution'] 

County,State,FIPS,Value_Pollutant: Benzene,Value_Pollutant: Formaldehyde,Value_Pollutant: Acetaldehyde,Value_Pollutant: Carbon tetrachloride,"Value_Pollutant: 1,3-butadiene",Value_Pollutant: Napthalene,Value_Pollutant: Ethylene oxide,Value_Pollutant: Diesel Particulate Matter,Data Comment_Pollutant: Benzene,Data Comment_Pollutant: Formaldehyde,Data Comment_Pollutant: Acetaldehyde,Data Comment_Pollutant: Carbon tetrachloride,"Data Comment_Pollutant: 1,3-butadiene",Data Comment_Pollutant: Napthalene,Data Comment_Pollutant: Ethylene oxide,Data Comment_Pollutant: Diesel Particulate Matter
str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str
"""Autauga""","""Alabama""",1001,0.15,1.84,1.49,0.51,0.01,0.02,0.0,0.11,"""""","""""","""""","""""","""""","""""","""""",""""""
"""Baldwin""","""Alabama""",1003,0.13,1.49,1.1,0.51,0.01,0.01,0.0,0.11,"""""","""""","""""","""""","""""","""""","""""",""""""
"""Barbour""","""Alabama""",1005,0.12,1.73,1.41,0.5,0.01,0.01,0.0,0.06,"""""","""""","""""","""""","""""","""""","""""",""""""
"""Bibb""","""Alabama""",1007,0.15,1.96,1.61,0.51,0.01,0.02,0.0,0.1,"""""","""""","""""","""""","""""","""""","""""",""""""
"""Blount""","""Alabama""",1009,0.16,1.63,1.19,0.5,0.01,0.01,0.0,0.12,"""""","""""","""""","""""","""""","""""","""""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Yabucoa""","""Puerto Rico""",72151,0.02,0.78,0.39,0.36,0.0,0.0,0.0,0.01,"""""","""""","""""","""""","""""","""""","""""",""""""
"""Yauco""","""Puerto Rico""",72153,0.03,0.95,0.43,0.35,0.0,0.0,0.0,0.02,"""""","""""","""""","""""","""""","""""","""""",""""""
"""St. Croix""","""Virgin Islands of the US""",78010,0.01,0.7,0.43,0.36,0.0,0.0,0.0,0.02,"""""","""""","""""","""""","""""","""""","""""",""""""


In [124]:
# Combining the directly asthma-related datasets and deleting the individual ones from memory to avoid clutter

df_clean['asthma'] = df_clean['emergency_asthma'].join(
    df_clean['asthma_hospitalization'],
    on=['County', 'State', 'FIPS'],
    how='left',
    coalesce=True)

del df_clean['emergency_asthma']
del df_clean['asthma_hospitalization']

## Medical Doctor and Respiratory Therapist counts per state

In [125]:
df_raw['md_per_state'].slice(1)

data.HRSA.gov,data.HRSA.gov_1,data.HRSA.gov_2,data.HRSA.gov_3,data.HRSA.gov_4
str,str,str,str,str
"""State/Territory""","""M.D., Active M.D. (County Leve…","""Population, All (County Level …","""Rate (per 100,000 population)""","""Source"""
"""Alabama""","""11632""","""4903185""","""237.23""","""AMA Physician Professional Dat…"
"""Alaska""","""1851""","""731545""","""253.03""","""AMA Physician Professional Dat…"
"""Arizona""","""17355""","""7278717""","""238.43""","""AMA Physician Professional Dat…"
"""Arkansas""","""6750""","""3017804""","""223.67""","""AMA Physician Professional Dat…"
…,…,…,…,…
"""Washington""","""21846""","""7614893""","""286.89""","""AMA Physician Professional Dat…"
"""West Virginia""","""4548""","""1792147""","""253.77""","""AMA Physician Professional Dat…"
"""Wisconsin""","""16105""","""5822434""","""276.6""","""AMA Physician Professional Dat…"


In [126]:
df_raw['resp_therapist_per_state']

data.HRSA.gov,data.HRSA.gov_1,data.HRSA.gov_2,data.HRSA.gov_3,data.HRSA.gov_4
str,str,str,str,str
"""Selected Filters: 2019 / Respi…",null,null,null,null
"""State/Territory""","""Respiratory Therapist, Total …","""Population, All (State Level F…","""Rate (per 100,000 population)""","""Source"""
"""Alabama""","""2257""","""4876250""","""46.29""","""American Community Survey PUMS…"
"""Alaska""",null,"""737068""",null,"""American Community Survey PUMS…"
"""Arizona""","""2619""","""7050299""","""37.15""","""American Community Survey PUMS…"
…,…,…,…,…
"""Washington""","""1654""","""7404107""","""22.34""","""American Community Survey PUMS…"
"""West Virginia""","""1089""","""1817305""","""59.92""","""American Community Survey PUMS…"
"""Wisconsin""","""1690""","""5790716""","""29.18""","""American Community Survey PUMS…"


In [127]:
# These datasets have the same issue from before, where the first row contains
# what should've been the headers.

descriptions_md = df_raw['md_per_state'].row(1)
descriptions_resp = df_raw['resp_therapist_per_state'].row(1)

print(descriptions_md)
print(descriptions_resp)

('State/Territory', 'M.D., Active M.D. (County Level File)', 'Population, All (County Level File)', 'Rate (per 100,000 population)', 'Source')
('State/Territory', 'Respiratory Therapist,  Total (State Level File)', 'Population, All (State Level File)', 'Rate (per 100,000 population)', 'Source')


In [128]:
# We zip the current column ID (header) with the description
# Both of these dataframes have very similar structuring so we can perform the steps for both as we go along
cols_to_keep = []
for col_id, desc in zip(df_raw['md_per_state'].columns, descriptions_md):

    cols_to_keep.append(pl.col(col_id).alias(desc))

# Performing the selection
df_clean['md_per_state'] = df_raw['md_per_state'].select(cols_to_keep).slice(2)

#####################################################################

cols_to_keep = []
for col_id, desc in zip(df_raw['resp_therapist_per_state'].columns, descriptions_resp):

    cols_to_keep.append(pl.col(col_id).alias(desc))

df_clean['resp_therapist_per_state'] = df_raw['resp_therapist_per_state'].select(cols_to_keep).slice(2)


# Renaming the State/Territory column to just State, 
# and the rate column to include what the rate is of to avoid clarity issues later

df_clean['md_per_state'] = df_clean['md_per_state'].rename({'State/Territory': 'State',
                                                            'Rate (per 100,000 population)': 'M.D per 100,000 population'})

df_clean['resp_therapist_per_state'] = df_clean['resp_therapist_per_state'].rename({'State/Territory': 'State',
                                                                                    'Rate (per 100,000 population)': 'Respiratory Therapist per 100,000 population'})

In [129]:
# Converting the columns into floats

text_cols = ['State', 'Source']

df_clean['md_per_state'] = convert_to_float(df_clean['md_per_state'], text_cols)

df_clean['resp_therapist_per_state'] = convert_to_float(df_clean['resp_therapist_per_state'], text_cols)



## County Health (and additonal) Measures

In [130]:
# A cursory glance of the .xls file reveals that the data in the '2019 County Health Rankings Data - v3.xls' file have multi-level headers,
# where the first row contains the category of data (low birthweight, infont mortality etc) and the second row contains the actual data
# type (counts, percentages, confidence intervals).

# This means we will need to take additional steps different from before to ensure there's no information loss or column name conflicts
df_raw['county_health_measures']

__UNNAMED__0,__UNNAMED__1,__UNNAMED__2,Premature death,__UNNAMED__4,__UNNAMED__5,__UNNAMED__6,__UNNAMED__7,__UNNAMED__8,__UNNAMED__9,Poor or fair health,__UNNAMED__11,__UNNAMED__12,__UNNAMED__13,Poor physical health days,__UNNAMED__15,__UNNAMED__16,__UNNAMED__17,Poor mental health days,__UNNAMED__19,__UNNAMED__20,__UNNAMED__21,Low birthweight,__UNNAMED__23,__UNNAMED__24,__UNNAMED__25,__UNNAMED__26,__UNNAMED__27,__UNNAMED__28,__UNNAMED__29,Adult smoking,__UNNAMED__31,__UNNAMED__32,__UNNAMED__33,Adult obesity,__UNNAMED__35,__UNNAMED__36,…,__UNNAMED__125,__UNNAMED__126,__UNNAMED__127,Social associations,__UNNAMED__129,__UNNAMED__130,Violent crime,__UNNAMED__132,__UNNAMED__133,Injury deaths,__UNNAMED__135,__UNNAMED__136,__UNNAMED__137,__UNNAMED__138,Air pollution - particulate matter,__UNNAMED__140,Drinking water violations,__UNNAMED__142,Severe housing problems,__UNNAMED__144,__UNNAMED__145,__UNNAMED__146,__UNNAMED__147,__UNNAMED__148,__UNNAMED__149,Driving alone to work,__UNNAMED__151,__UNNAMED__152,__UNNAMED__153,__UNNAMED__154,__UNNAMED__155,__UNNAMED__156,Long commute - driving alone,__UNNAMED__158,__UNNAMED__159,__UNNAMED__160,__UNNAMED__161
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""FIPS""","""State""","""County""","""Years of Potential Life Lost R…","""95% CI - Low""","""95% CI - High""","""Quartile""","""YPLL Rate (Black)""","""YPLL Rate (Hispanic)""","""YPLL Rate (White)""","""% Fair/Poor""","""95% CI - Low""","""95% CI - High""","""Quartile""","""Physically Unhealthy Days""","""95% CI - Low""","""95% CI - High""","""Quartile""","""Mentally Unhealthy Days""","""95% CI - Low""","""95% CI - High""","""Quartile""","""Unreliable""","""% LBW""","""95% CI - Low""","""95% CI - High""","""Quartile""","""% LBW (Black)""","""% LBW (Hispanic)""","""% LBW (White)""","""% Smokers""","""95% CI - Low""","""95% CI - High""","""Quartile""","""% Obese""","""95% CI - Low""","""95% CI - High""",…,"""95% CI - Low""","""95% CI - High""","""Quartile""","""# Associations""","""Association Rate""","""Quartile""","""Annual Average Violent Crimes""","""Violent Crime Rate""","""Quartile""","""# Injury Deaths""","""Injury Death Rate""","""95% CI - Low""","""95% CI - High""","""Quartile""","""Average Daily PM2.5""","""Quartile""","""Presence of violation""","""Quartile""","""% Severe Housing Problems""","""95% CI - Low""","""95% CI - High""","""Severe Housing Cost Burden""","""Overcrowding""","""Inadequate Facilities""","""Quartile""","""% Drive Alone""","""95% CI - Low""","""95% CI - High""","""Quartile""","""% Drive Alone (Black)""","""% Drive Alone (Hispanic)""","""% Drive Alone (White)""","""# Workers who Drive Alone""","""% Long Commute - Drives Alone""","""95% CI - Low""","""95% CI - High""","""Quartile"""
"""01001""","""Alabama""","""Autauga""","""8824.0571232""","""7935.3258295""","""9712.7884169""","""1""","""10471.252986""",null,"""8706.6588323""","""18.411124355""","""17.844049129""","""19.032316178""","""1""","""4.200577983""","""4.035998438""","""4.367509494""","""1""","""4.306739283""","""4.122621167""","""4.490923426""","""1""",null,"""8.475719424""","""7.657197852""","""9.294240997""","""1""","""12.620320856""",null,"""7.397260274""","""19.124658497""","""18.479562407""","""19.784979123""","""1""","""37.5""","""32.1""","""43.2""",…,"""19.206010479""","""30.194183248""","""1""","""70""","""12.631730908""","""2""","""148.5""","""272.28222006""","""2""","""205""","""74.031808398""","""63.9""","""84.2""","""2""","""11.7""","""4""","""No""","""1""","""14.954645747""","""12.349507627""","""17.559783866""","""13.205222961""","""2.45158127""","""0.63741113""","""3""","""85.965056526""","""83.029265858""","""88.900847195""","""3""","""82.722036981""",null,"""83.710021322""","""20911""","""3

In [131]:
# Since we have two dataframes with this format and the process is somewhat convoluted, its better to define a function

def flatten_hierarchical_headers(df, separator =  ' => '):
    """
    Takes a dataframe with hierarchical headers (where Row 1 is categories and 
    Row 2 is sub-columns) and flattens them into a single header row.
    """
    # Processing top-level headers (Categories) to handle '__UNNAMED__'
    raw_categories = df.columns
    clean_categories = []
    current_category = None

    for col in raw_categories:
        # If the category name is valid, updates the current name. If UNNAMED, uses previous name.
        if not col.startswith('__UNNAMED__'):
            current_category = col
        
        # Handling the case where the very first column might be unnamed 
        # (was not the case in the first file but better to be safe)
        clean_categories.append(current_category if current_category else '')

    # Extracting sub-headers from the first row of data
    # We convert nulls to empty strings to avoid errors during string formatting
    sub_headers = [val if val is not None else '' for val in df.row(0)]

    # Generating new column names
    new_columns = []
    for cat, sub in zip(clean_categories, sub_headers):
        # Stripping the whitespace to ensure clean comparisons
        cat = str(cat).strip()
        sub = str(sub).strip()
        
        if not cat:
            # If no category exists, uses the sub-header
            new_columns.append(sub)
        elif not sub or cat == sub:
            # If sub is empty, or if Category and Sub are identical (e.g. 'County'), 
            # just uses the category
            new_columns.append(cat)
        else:
            # Combining them
            new_columns.append(f"{cat}{separator}{sub}")

    # Removing the first row (the old sub-headers) and applying new column names
    df_cleaned = df.slice(1)
    df_cleaned.columns = new_columns
    
    return df_cleaned

In [132]:
df_clean['county_health_measures'] = flatten_hierarchical_headers(df_raw['county_health_measures'], ' => ')
df_clean['county_additional_measures'] = flatten_hierarchical_headers(df_raw['county_additional_measures'], ' => ')

In [133]:
# There are a few columns in this dataset that don't provide additional context that the
# actual data features don't, like the quartile info.

# Dropping the confidence interval, quartile and margin of error columns
banned_terms = ['Quartile', '95% CI', 'MOE']

# Filtering the list of column names
# We keep a column ONLY if it does NOT contain any of the banned terms
clean_columns_health = [
    c for c in df_clean['county_health_measures'].columns 
    if not any(term in c for term in banned_terms)
]

df_clean['county_health_measures'] = df_clean['county_health_measures'].select(clean_columns_health)


clean_columns_additional = [
    c for c in df_clean['county_additional_measures'].columns 
    if not any(term in c for term in banned_terms)
]

df_clean['county_additional_measures'] = df_clean['county_additional_measures'].select(clean_columns_additional)


In [134]:
# These datasets also have data stored as both aggregates and percentages denoted with '#' and '%' respectively.
# We can drop the aggregate columns and keep only the percentage since population differences between counties
# makes percentages more useful for comparison.

def drop_redundant_aggregates(df):
    """
    Scans column names for pairs where one uses '#' (count) and the other uses '%' (rate).
    Drops the '#' version if and only if the '%' version exists.
    """
    all_cols = set(df.columns)
    cols_to_drop = []
        
    for col in df.columns:
        # We only care about columns that explicitly use the '#' symbol
        if '#' in col:
            # Constructing the theoretical rate counterpart
            # This handles cases like 'Unemployment#' -> 'Unemployment%'
            # or 'Values (#)' -> 'Values (%)'
            rate_counterpart = col.replace('#', '%')
            
            # CRITICAL CHECK: Does the rate version actually exist?
            if rate_counterpart in all_cols:
                cols_to_drop.append(col)
                print(f'  - Dropping "{col}" (redundant to "{rate_counterpart}")')
    
    if not cols_to_drop:
        print('  No redundant count/rate pairs found.')
        return df
        
    print(f'Dropped {len(cols_to_drop)} redundant count columns.')
    return df.drop(cols_to_drop)

#################################

df_clean['county_health_measures'] = drop_redundant_aggregates(df_clean['county_health_measures'])
df_clean['county_additional_measures'] = drop_redundant_aggregates(df_clean['county_additional_measures'])


  - Dropping "Uninsured => # Uninsured" (redundant to "Uninsured => % Uninsured")
  - Dropping "Some college => # Some College" (redundant to "Some college => % Some College")
  - Dropping "Unemployment => # Unemployed" (redundant to "Unemployment => % Unemployed")
  - Dropping "Children in single-parent households => # Single-Parent Households" (redundant to "Children in single-parent households => % Single-Parent Households")
Dropped 4 redundant count columns.
  - Dropping "Food insecurity => # Food Insecure" (redundant to "Food insecurity => % Food Insecure")
  - Dropping "Limited access to healthy foods => # Limited Access" (redundant to "Limited access to healthy foods => % Limited Access")
  - Dropping "Uninsured adults => # Uninsured" (redundant to "Uninsured adults => % Uninsured")
  - Dropping "Uninsured children => # Uninsured" (redundant to "Uninsured children => % Uninsured")
  - Dropping "Homeownership => # Homeowners" (redundant to "Homeownership => % Homeowners")
  - Dro

In [135]:
df_clean['county_additional_measures']

FIPS,State,County,Life expectancy => Life Expectancy,Life expectancy => Life Expectancy (Black),Life expectancy => Life Expectancy (Hispanic),Life expectancy => Life Expectancy (White),Premature age-adjusted mortality => # Deaths,Premature age-adjusted mortality => Age-Adjusted Mortality,Premature age-adjusted mortality => Age-Adjusted Mortality (Black),Premature age-adjusted mortality => Age-Adjusted Mortality (Hispanic),Premature age-adjusted mortality => Age-Adjusted Mortality (White),Child mortality => # Deaths,Child mortality => Child Mortality Rate,Child mortality => Child Mortality Rate (Black),Child mortality => Child Mortality Rate (Hispanic),Child mortality => Child Mortality Rate (White),Infant mortality => # Deaths,Infant mortality => Infant Mortality Rate,Infant mortality => Infant Mortality Rate (Black),Infant mortality => Infant Mortality Rate (Hispanic),Infant mortality => Infant Mortality Rate (White),Frequent physical distress => % Frequent Physical Distress,Frequent mental distress => % Frequent Mental Distress,Diabetes prevalence => % Diabetic,HIV prevalence => # HIV Cases,HIV prevalence => HIV Prevalence Rate,Food insecurity => % Food Insecure,Limited access to healthy foods => % Limited Access,Drug overdose deaths => # Drug Overdose Deaths,Drug overdose deaths => Drug Overdose Mortality Rate,Motor vehicle crash deaths => # Motor Vehicle Deaths,Motor vehicle crash deaths => MV Mortality Rate,Insufficient sleep => % Insufficient Sleep,Uninsured adults => % Uninsured,Uninsured children => % Uninsured,Other primary care providers => Other PCP Rate,Other primary care providers => Other PCP Ratio,Disconnected youth => % Disconnected Youth,Median household income => Household Income,Median household income => Household income (Black),Median household income => Household income (Hispanic),Median household income => Household income (White),Children eligible for free or reduced price lunch => % Free or Reduced Lunch,Residential segregation - black/white => Segregation index,Residential segregation - non-white/white => Segregation Index,Homicides => Homicide Rate,Firearm fatalities => # Firearm Fatalities,Firearm fatalities => Firearm Fatalities Rate,Homeownership => % Homeowners,Severe housing cost burden => # Households with Severe Cost Burden,Severe housing cost burden => % Severe Housing Cost Burden,Demographics => Population,Demographics => % < 18,Demographics => % 65 and over,Demographics => % African American,Demographics => % American Indian/Alaskan Native,Demographics => % Asian,Demographics => % Native Hawaiian/Other Pacific Islander,Demographics => % Hispanic,Demographics => % Non-Hispanic White,Demographics => % Not Proficient in English,Demographics => % Female,Demographics => % Rural
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""01001""","""Alabama""","""Autauga""","""76.330589011""","""74.600627791""",null,"""76.337219427""","""815""","""438.7""","""532.97759503""",null,"""432.17584731""","""29""","""53.279441484""","""92.748735245""",null,"""45.703839122""","""34""","""7.699275362""",null,null,null,"""12.824434466""","""12.638357067""","""14.2""","""103""","""225.5""","""13.4""","""11.991125""","""16""","""9.623076137""","""79""","""20.37715798""","""35.905406064""","""10.961103842""","""2.476190476""","""30.62842""","""3265:1""","""8.011653314""","""58343""","""28595""","""89359""","""61380""","""47.931664339""","""29.945392802""","""27.14756927""","""5.2""","""49""","""17.7""","""73.287736297""","""2569""","""12.645205749""","""55504""","""23.937013549""","""15.119631018""","""19.254468146""","""0.475641395""","""1.279187086""","""0.104496973""","""2.857451715""","""74.47391179""","""0.828548306""","""51.342245604""","""42.002162321"""
"""01003""","""Alabama""","""Baldwin""","""78.5994976

## Social Vulnerability Index

In [136]:
# These datasets use codenames for column labels which is not humanly readable. The corresponding descriptions
# are stored in the documentation pdf, inside a table. To avoid having to manually assign them, the easiest approach
# is to extract the table from the pdf

df_raw['svi_2018']#.row(0)

ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,…,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,GeoLevel,Comparison
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ST""","""STATE""","""ST_ABBR""","""COUNTY""","""FIPS""","""""","""""","""E_TOTPOP""","""M_TOTPOP""","""E_HU""","""M_HU""","""E_HH""","""M_HH""","""E_POV""","""M_POV""","""E_UNEMP""","""M_UNEMP""","""E_PCI""","""M_PCI""","""E_NOHSDP""","""M_NOHSDP""","""E_AGE65""","""M_AGE65""","""E_AGE17""","""M_AGE17""","""E_DISABL""","""M_DISABL""","""E_SNGPNT""","""M_SNGPNT""","""E_MINRTY""","""M_MINRTY""","""E_LIMENG""","""M_LIMENG""","""E_MUNIT""","""M_MUNIT""","""E_MOBILE""","""M_MOBILE""",…,"""RPL_THEME3""","""EPL_MUNIT""","""EPL_MOBILE""","""EPL_CROWD""","""EPL_NOVEH""","""EPL_GROUPQ""","""SPL_THEME4""","""RPL_THEME4""","""SPL_THEMES""","""RPL_THEMES""","""F_POV""","""F_UNEMP""","""F_PCI""","""F_NOHSDP""","""F_THEME1""","""F_AGE65""","""F_AGE17""","""F_DISABL""","""F_SNGPNT""","""F_THEME2""","""F_MINRTY""","""F_LIMENG""","""F_THEME3""","""F_MUNIT""","""F_MOBILE""","""F_NOVEH""","""""","""F_GROUPQ""","""F_THEME4""","""F_TOTAL""","""E_UNINSUR""","""M_UNINSUR""","""EP_UNINSUR""","""MP_UNINSUR""","""E_DAYPOP""","""""",""""""
"""01""","""ALABAMA""","""AL""","""Autauga""","""01001""","""Autauga County, Alabama""","""594.4434589""","""55200""","""0""","""23315""","""71""","""21115""","""383""","""8422""","""1137""","""1065""","""257""","""29372""","""2306""","""4204""","""475""","""8050""","""75""","""13369""","""32""","""10465""","""729""","""1586""","""319.9""","""13788""","""59.0""","""426""","""205.9""","""886""","""308.7""","""4279""","""469""",…,"""0.5947""","""0.6017""","""0.7408""","""0.2964""","""0.4846""","""0.1525""","""2.276""","""0.3741""","""7.0688""","""0.4354""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""3875""","""508""","""7.1""","""0.9""","""37301""","""County""","""National"""
"""01""","""ALABAMA""","""AL""","""Baldwin""","""01003""","""Baldwin County, Alabama""","""1589.7930067""","""208107""","""0""","""111945""","""206""","""78622""","""1183""","""21653""","""1765""","""4343""","""620""","""31203""","""716""","""14310""","""1211""","""40665""","""69""","""45677""","""0""","""28863""","""1217""","""4508""","""581.4""","""35339""","""227.0""","""1068""","""323.6""","""20492""","""1203.0""","""13075""","""831""",…,"""0.4358""","""0.9713""","""0.5339""","""0.2604""","""0.1328""","""0.3018""","""2.2002""","""0.3359""","""5.7034""","""0.2162""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""1""","""1""","""20864""","""1646""","""10.2""","""0.8""","""195677""","""County""","""National"""
"""01""","""ALABAMA""","""AL""","""Barbour""","""01005""","""Barbour County, Alabama""","""885.0016356""","""25782""","""0""","""11937""","""123""","""9186""","""280""","""6597""","""661""","""918""","""201""","""18461""","""819""","""4901""","""333""","""4634""","""29""","""5436""","""38""","""5078""","""387""","""1174""","""187.9""","""13884""","""22.0""","""398""","""165.2""","""152""","""79.3""","""3484""","""23

In [137]:
# The codenames and their corresponding description are stored in one table that spans multiple pages,
# and the features stored here are color coded according to a table before this table where each color
# represents one category of statistic:
# Household Composition/Disability, Socioeconomic, Minory Status/Language, Housing Type/Transportation or General
# To that end, we will need to not only extract the text in the second table, but also the colors and what the 
# colors represent in the first table. This is a computationally complex task that we're better off handling
# in multiple steps

import pdfplumber

# The background color extraction works separately from the text extraction, so for easier troubleshooting's sake
# it gets its own function (to be used inside the text extraction function)
def get_bbox_color(page, bbox):
    """
    Analyzes a specific bounding box to find the background color.
    Shrinks the analysis area slightly to avoid picking up black grid lines.
    """
    # Ensuring the bounding box has positive width/height
    if (bbox[2] - bbox[0] <= 4) or (bbox[3] - bbox[1] <= 4):
        return None

    # Shrinking the color detection area significantly to hit the 'center' of the cell
    color_padding = 2
    safe_bbox = (
        bbox[0] + color_padding, 
        bbox[1] + color_padding, 
        bbox[2] - color_padding, 
        bbox[3] - color_padding
    )
    
    try:
        cropped = page.crop(safe_bbox)
        rects = cropped.rects
        
        if not rects:
            return None
        
        largest_rect = max(rects, key=lambda x: x['width'] * x['height'])
        color = largest_rect.get('non_stroking_color')
        
        return str(tuple(color)) if color else None
    except Exception:
        return None

In [138]:
def extract_tables_with_colors(pdf_path, table_settings=None):
    """
    This function takes a path to a .pdf file and returns a dictionary containing dictionaries of dataframe pairs
    where one dataframe stores the text and the other stores colors.
    """
    all_logical_tables = []
    
    # From testing, these settings worked best so they're left as the default, but the function can be passed
    # custom settings for the table finding operation
    DEFAULT_TABLE_SETTINGS = {
        'vertical_strategy': 'lines',
        'horizontal_strategy': 'lines',
        'join_x_tolerance': 3,
        'join_y_tolerance': 3,
        'snap_x_tolerance': 5.5, 
        'snap_y_tolerance': 5, 
        'intersection_tolerance': 4.0,
        #'edge_min_length': 200,
        'text_x_tolerance': 1,
        'text_y_tolerance': 2
    }

    if table_settings is None:
        table_settings = DEFAULT_TABLE_SETTINGS

    with pdfplumber.open(pdf_path) as pdf:

        ################# Forming the current table #################
        
        current_logical_table = {
            'headers': None,
            'data_rows': [], 
            'color_rows': []  
        }
        
        # Iterating per page
        for page in pdf.pages:
            # Detecting the grid structure
            tables = page.find_tables(table_settings = table_settings)
            
            # Iterating per table on the page
            for table in tables:
                page_text_rows = []
                page_color_rows = []
                
                # Iterating per row on the table
                for row in table.rows:
                    row_texts = []
                    row_colors = []
                    
                    # Iterating per cell on the row
                    for cell in row.cells:
                        if cell is None:
                            row_texts.append('')
                            row_colors.append(None)
                            continue

                        # Forming the bounding box of the cell from grid lines
                        bbox = (cell[0], cell[1], cell[2], cell[3])
                        
                        # Text extraction steps:
                        # Shrinking the bounding box to prevent bleed-through from rows above/below 
                        # Some of the text is so densely packed inside the cell pdfplumber has trouble
                        # differentiating between adjacent rows otherwise
                        v_pad = 1.5
                        h_pad = 1.0
                        
                        if (bbox[3] - bbox[1] > 2*v_pad) and (bbox[2] - bbox[0] > 2*h_pad):
                            text_crop_bbox = (
                                bbox[0] + h_pad, 
                                bbox[1] + v_pad, 
                                bbox[2] - h_pad, 
                                bbox[3] - v_pad
                            )
                        else:
                            text_crop_bbox = bbox

                        # Increasing Tolerance to fix spacing issues (to avoid recognizing non-existent spaces between letters)
                        try:
                            cropped_page = page.crop(text_crop_bbox)
                            # x_tolerance=3 joins letters that are slightly apart
                            text = cropped_page.extract_text(x_tolerance=3, y_tolerance=3)
                        except Exception:
                            text = ''

                        row_texts.append(text.strip() if text else '')
                        
                        # Extracting cell color with the previously defined function
                        color = get_bbox_color(page, bbox)
                        row_colors.append(color)
                        
                    page_text_rows.append(row_texts)
                    page_color_rows.append(row_colors)

                if not page_text_rows:
                    continue

                # Merging the rows to form the tables
                
                headers = page_text_rows[0]
                
                # Case 1: Start of a brand new table logic
                if current_logical_table['headers'] is None:
                    current_logical_table['headers'] = headers
                    current_logical_table['data_rows'] = page_text_rows[1:]
                    current_logical_table['color_rows'] = page_color_rows[1:]
                    continue

                # Case 2: Continuation of the existing table
                # Since there are a few cases where the text of one cell does not fit in a page and
                # has to continue onto a new row in the next page, we need to define a specific check to catch
                # these exceptions. The first column is always full in the original table otherwise, so that's
                # our lead to actually merge such rows with the last row of the previous table instead
                # The function also compares headers just in case
                if headers == current_logical_table['headers']:
                    rows_to_add = page_text_rows[1:]
                    colors_to_add = page_color_rows[1:]
                    
                    start_index = 0
                    
                    first_row = rows_to_add[0] if rows_to_add else None
                    
                    # 'Cut off' logic: 
                    # If Col 0 (Code) is empty, but Col 1 (Description) has text,
                    # merges Col 1 into the previous row.
                    if first_row and len(first_row) > 1 and (first_row[0] == '') and first_row[1]:
                        if current_logical_table['data_rows']:
                            last_idx = len(current_logical_table['data_rows']) - 1
                            prev_text = current_logical_table['data_rows'][last_idx][1]
                            new_text = first_row[1]
                            
                            # Combining the text
                            current_logical_table['data_rows'][last_idx][1] = f'{prev_text} {new_text}'
                            
                            # Skiping this row since we merged it
                            start_index = 1
                    
                    current_logical_table['data_rows'].extend(rows_to_add[start_index:])
                    current_logical_table['color_rows'].extend(colors_to_add[start_index:])
                    
                else:
                    # Case 3: A completely different table detected
                    # This technically should happen only once in this pdf but table extraction is still
                    # prone to issues so its better to play it safe here and perform quality control and merging
                    # later on in the case of misrecognized headers
                    all_logical_tables.append(current_logical_table)
                    current_logical_table = {
                        'headers': headers,
                        'data_rows': page_text_rows[1:],
                        'color_rows': page_color_rows[1:]
                    }

        # Building the very last table
        if current_logical_table['headers']:
            all_logical_tables.append(current_logical_table)

    # Converting the result to Polars dataframe
    final_output = {}
    
    for i, tbl in enumerate(all_logical_tables):
        raw_headers = tbl['headers']
        data = tbl['data_rows']
        colors = tbl['color_rows']
        
        ################# Sanitizing #################

        clean_headers = []
        if raw_headers:
            for idx, h in enumerate(raw_headers):

                # Fallback header definition in case pdfplumber fails (we will handle these later)
                if h is None or h.strip() == '':
                    clean_headers.append(f'Column_{idx+1}')
                else:
                    # Flattening newlines in headers, pdfplumber detects a lot of these since the second
                    # table has very compact columns
                    clean_headers.append(h.replace('\n', ' ').strip())
        else:
            # Fallback if find_tables returned a structure with no text at all in row 0
            if data:
                clean_headers = [f'Column_{x+1}' for x in range(len(data[0]))]
            else:
                clean_headers = ['Column_1']

        # Normalizing Data Rows to match Header Length
        # This prevents the Polars 'Shape Error' if rows have different cell counts
        expected_len = len(clean_headers)
        
        normalized_data = []
        for row in data:
            # Padding with empty strings or slice to match header width
            new_row = row[:expected_len] + [''] * (expected_len - len(row))
            normalized_data.append(new_row)
            
        normalized_colors = []
        for row in colors:
            new_row = row[:expected_len] + [None] * (expected_len - len(row))
            normalized_colors.append(new_row)

        try:
            df_text = pl.DataFrame(normalized_data, schema=clean_headers, orient='row')
            df_color = pl.DataFrame(normalized_colors, schema=clean_headers, orient='row')
            
            final_output[f'Table_{i+1}'] = {
                'text': df_text,
                'colors': df_color
            }
        except Exception as e:
            print(f'Error creating Table_{i+1}: {e}')
        
    return final_output

In [139]:
# These are the same pdfplumber settings as the defaults set inside the function
table_settings = {
    'vertical_strategy': 'lines',
    'horizontal_strategy': 'lines',
    'join_x_tolerance': 3,
    'join_y_tolerance': 3,
    'snap_x_tolerance': 5.5, 
    'snap_y_tolerance': 5, 
    'intersection_tolerance': 4.0,
    #'edge_min_length': 200,
    'text_x_tolerance': 1,
    'text_y_tolerance': 2
}

svi2018_pdf = r'C:\Users\mpola\asthma analysis\files\parsed files\svi_2018_Documentation.pdf'

result = extract_tables_with_colors(svi2018_pdf, table_settings)
print(result['Table_2']['text'])
print(result['Table_2']['colors'])
print(result['Table_3']['text'])

for row in result['Table_3']['text'].iter_rows():
    print(row)

shape: (18, 9)
┌────────────┬────────────┬───────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ 2018       ┆ 2018 DESCR ┆ THEME ┆ CENSUS or  ┆ … ┆ 2018      ┆ CALCULATI ┆ NOTES     ┆ 2016      │
│ VARIABLE   ┆ IPTION     ┆ ---   ┆ SVI        ┆   ┆ TABLE     ┆ ON DESCRI ┆ ---       ┆ TABLE     │
│ NAME       ┆ ---        ┆ str   ┆ TABLE(S)   ┆   ┆ FIELD CAL ┆ PTION     ┆ str       ┆ FIELD CAL │
│ ---        ┆ str        ┆       ┆ ---        ┆   ┆ CULATION  ┆ ---       ┆           ┆ CULATION  │
│ str        ┆            ┆       ┆ str        ┆   ┆ ---       ┆ str       ┆           ┆ i…        │
│            ┆            ┆       ┆            ┆   ┆ str       ┆           ┆           ┆ ---       │
│            ┆            ┆       ┆            ┆   ┆           ┆           ┆           ┆ str       │
╞════════════╪════════════╪═══════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ ST         ┆ State-leve ┆       ┆ SVI        ┆ … ┆ FIPS      ┆ In Excel, ┆

In [140]:
# While the text detection is mostly servicable, the current function sometimes fails when it
# comes to header recognition and reads parts of the header as separate rows, and also makes the same
# error with a few of the rows. We're fortunate that the first row is brief and consistent enough
# to act as an anchor to systematically check the rows and collapse them as needed.

# At this point it is easier to define a new function to clean the existing dictionaries
# than it is to further troubleshoot the existing function. We handle the line break
# issues here while we're at it
def normalize_text(text):
    """Helper to strip whitespace and lower-case for comparison."""
    if text is None:
        return ''
    return str(text).lower().replace(' ', '').replace('\n', '').strip()

def clean_and_merge_tables(tables_dict, target_headers):
    """
    This function post-processes the dictionary from the previous step to clean and merge all the
    tables that should be one table, taking a list of headers as both the basis for the headers and the
    column count of the final table. Any tables within the dictionary that don't have as many columns
    as the list has string entries will be ignored.
    
    Args:
        tables_dict: Dictionary with keys 'Table_X' containing {'text': df, 'colors': df} (the output of the previous function)
        target_headers: List of strings (e.g. ['Code', 'Description'])
        
    Returns:
        dict: {'text': combined_df, 'colors': combined_df}
    """
    # Defining the 'end goal' headers list and the column count for the final dataframe
    target_width = len(target_headers)
    target_fingerprint = normalize_text(''.join(target_headers))
    
    final_text_rows = []
    final_color_rows = []
    
    # Iterating per table in the dictionary
    for table_name, data in tables_dict.items():
        df_text = data['text']
        df_color = data['colors']
        
        # Filtering (skipping the tables that don't match the column count given by the list)
        if df_text.width != target_width:
            print(f'Skipping {table_name}: Width {df_text.width} does not match target {target_width}')
            continue
            
        # Converting to Python lists for mutable manipulation
        # Replacing None with '' to avoid concatenation errors
        text_rows = [[str(c) if c is not None else '' for c in row] for row in df_text.rows()]
        color_rows = [[c for c in row] for row in df_color.rows()]
        
        # Getting initial headers from the DataFrame columns
        # If headers are 'Column_X' (the default name given to headers with no text recognized), 
        # treat them as empty for reconstruction purposes
        current_headers = []
        for col in df_text.columns:
            if col.startswith('Column_'):
                current_headers.append('')
            else:
                current_headers.append(col)
        
        ################# Header stitching #################
        data_start_index = 0
        header_matched = False
        
        # Checking if the dataframe columns already match
        current_fingerprint = normalize_text(''.join(current_headers))
        if current_fingerprint == target_fingerprint:
            header_matched = True
        else:
            # Iteratively consuming rows to build the header (since the header can also
            # end up split into multiple rows rather than just the first row)
            temp_headers = list(current_headers)
            
            for i, row in enumerate(text_rows):
                # Checking if this row is the header row itself (e.g. Page 2 repeating headers)
                row_fingerprint = normalize_text(''.join(row))
                if row_fingerprint == target_fingerprint:
                    data_start_index = i + 1
                    header_matched = True
                    break

                # Combining rows without an entry in the first column onto the headers
                is_continuation = (row[0].strip() == '')
                
                # Updating our temp headers
                for col_idx in range(target_width):
                    # If it's a placeholder 'Column_X', it gets replaced entirely. Otherwise it gets appended onto.
                    if 'Column_' in temp_headers[col_idx] or temp_headers[col_idx] == '':
                         temp_headers[col_idx] = row[col_idx]
                    else:
                        if row[col_idx].strip():
                            temp_headers[col_idx] += ' ' + row[col_idx]
                
                # Checking match
                check_fingerprint = normalize_text(''.join(temp_headers))
                if check_fingerprint == target_fingerprint:
                    data_start_index = i + 1
                    header_matched = True
                    break
                
                # If we hit a row that HAS a first column entry, but we haven't matched headers yet,
                # we have a problem. The logic dictates we only consume empty-first-col rows.
                if not is_continuation:
                    # Sanity check in case the row IS the header but was split
                    # If not, we have to abort or assume this is data and the header logic failed.
                    break

        if not header_matched:
            # Sometimes headers are just missing on page 3+.
            # We will assume if we hit a valid Code in col 1, data has started.
            print(f'Warning: Precise header match not found for {table_name}. Attempting to process data.')
            pass
            
        ################# Row collapsing #################

        # Collapsing rows without first entries onto the first row with a first entry
        # This solves the problem of row entries that are too dense being recognized as
        # separate rows
        
        subset_text = text_rows[data_start_index:]
        subset_color = color_rows[data_start_index:]
        
        if not subset_text:
            continue

        current_valid_row = None
        current_valid_color = None

        for i, row in enumerate(subset_text):
            col_1_val = row[0].strip()
            
            # Logic: Is this a new ID (e.g. 'E_NOHSDP')?
            if col_1_val:
                # Saving the PREVIOUS valid row if it exists
                if current_valid_row:
                    final_text_rows.append(current_valid_row)
                    final_color_rows.append(current_valid_color)
                
                # Starting a NEW valid row
                current_valid_row = list(row)
                current_valid_color = list(subset_color[i])
                
            else:
                # Logic: This is a continuation row (empty first col)
                if current_valid_row is not None:
                    # Appending text to the current valid row (skipping col 0)
                    for c in range(1, target_width):
                        text_fragment = row[c].strip()
                        if text_fragment:
                            if current_valid_row[c]:
                                current_valid_row[c] += ' ' + text_fragment
                            else:
                                current_valid_row[c] = text_fragment
                    
                    # Prioritizing the colors in the row where the first column has an entry
                    # So we generally DO NOT update the color from the continuation row.
                    # We keep current_valid_color as is.
                    # When cells are color-coded it is consistent across the row, and
                    # since the first column is the most 'secure' column in terms of 
                    # error likelihood, we use it as the color anchor as well
                else:
                    # Corner case: We found a continuation row before ANY valid row.
                    # This implies it belongs to the previous PAGE'S last row 
                    # (the aforementioned overflow issue that exists in the pdf itself rather than our reading of it)
                    # We can append it to the global list's last entry.
                    if final_text_rows:
                        last_global_row = final_text_rows[-1]
                        for c in range(1, target_width):
                            text_fragment = row[c].strip()
                            if text_fragment:
                                if last_global_row[c]:
                                    last_global_row[c] += ' ' + text_fragment
                                else:
                                    last_global_row[c] = text_fragment
                    # If no global row exists yet, this data is effectively orphaned (or header junk)
                    # This is an extreme edge case that is luckily irrelevant to us

        # Appending the very last row being built
        if current_valid_row:
            final_text_rows.append(current_valid_row)
            final_color_rows.append(current_valid_color)

    ################## FInal output generation #################
    
    if not final_text_rows:
        raise ValueError('No valid rows were extracted matching the target headers.')

    df_final_text = pl.DataFrame(
        final_text_rows, 
        schema=target_headers, 
        orient='row'
    )
    
    df_final_colors = pl.DataFrame(
        final_color_rows, 
        schema=target_headers, 
        orient='row'
    )
    
    return {
        'text': df_final_text,
        'colors': df_final_colors
    }

######################################################

# We can use the second detected table's headers as our reference since the detection was accurate here
headers = result['Table_2']['text'].columns

svi2018_table = clean_and_merge_tables(result, headers)
svi2018_colors = result['Table_1']

# Previewing
print(svi2018_table['text'])
print(svi2018_table['colors'])

Skipping Table_1: Width 1 does not match target 9
shape: (124, 9)
┌────────────┬────────────┬───────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ 2018       ┆ 2018 DESCR ┆ THEME ┆ CENSUS or  ┆ … ┆ 2018      ┆ CALCULATI ┆ NOTES     ┆ 2016      │
│ VARIABLE   ┆ IPTION     ┆ ---   ┆ SVI        ┆   ┆ TABLE     ┆ ON DESCRI ┆ ---       ┆ TABLE     │
│ NAME       ┆ ---        ┆ str   ┆ TABLE(S)   ┆   ┆ FIELD CAL ┆ PTION     ┆ str       ┆ FIELD CAL │
│ ---        ┆ str        ┆       ┆ ---        ┆   ┆ CULATION  ┆ ---       ┆           ┆ CULATION  │
│ str        ┆            ┆       ┆ str        ┆   ┆ ---       ┆ str       ┆           ┆ i…        │
│            ┆            ┆       ┆            ┆   ┆ str       ┆           ┆           ┆ ---       │
│            ┆            ┆       ┆            ┆   ┆           ┆           ┆           ┆ str       │
╞════════════╪════════════╪═══════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ ST         ┆ State-leve

In [141]:
# Confirming the final row count of the table matches the column count of the dataframe
print(len(df_raw['svi_2018'].columns))

for i in df_raw['svi_2018'].columns:
    if i not in svi2018_table['text'].to_series(0).to_list():
        print(f'{i} was found in the dataframe but not the table')

for i in svi2018_table['text'].to_series(0).to_list():
    if i not in df_raw['svi_2018'].columns:
        print(f'{i} was found in the table but not the dataframe')

125
GeoLevel was found in the dataframe but not the table
Comparison was found in the dataframe but not the table
STCNTY was found in the table but not the dataframe


In [142]:
print('Unique entries of the Comparison column:')
for i in df_raw['svi_2018'].to_series(-1).unique():
    print(i)

print('Unique entries of the GeoLevel column:')
for i in df_raw['svi_2018'].to_series(-2).unique():
    print(i)

Unique entries of the Comparison column:
National

Unique entries of the GeoLevel column:

County


In [143]:
# In the end, the only features that don't match between the table and the dataframe are:
# GeoLevel, which is always County
# Comparison, which is always National (the blank entry is a header misread by Polars' read_csv that'll be fixed)
# STCNTY, which is the FIPS code mislabeled between the .csv file and the .pdf table

# Therefore, we can move forward with using the cleaned and merged table we've created via the .pdf to
# assemble a rename mapping for our dataframe without issue

def create_dataset_mapping(category_table, glossary_table):
    """
    Creates a mapping dictionary from identifier codes to formatted descriptions.
    
    Args:
        category_table (dict): {'text': pl.DataFrame, 'colors': pl.DataFrame} for the color legend.
        glossary_table (dict): {'text': pl.DataFrame, 'colors': pl.DataFrame} for the codes/descriptions.
        
    Returns:
        dict: { 'E_POV': '(Socioeconomic) Persons below poverty... [E_POV]', ... }
    """

    # General text cleaner function 
    # (should not be needed due to earlier steps taken but better safe than sorry)
    def clean_text(s):
        if not s: 
            return ''
        # Replacing newlines with a single space
        s = s.replace('\n', ' ')
        # Replacing multiple spaces (including tabs) with a single space
        s = re.sub(r'\s+', ' ', s)
        return s.strip()


    ################# Building the color legend map #################
    # Maps the string representation of the color to the category name
    # using the first table from the pdf (the first input of the function)
    color_to_category = {}
    
    cat_texts = category_table['text'].rows()
    cat_colors = category_table['colors'].rows()
    
    for text_row, color_row in zip(cat_texts, cat_colors):
        raw_name = clean_text(text_row[0])
        
        # Specific fix: Removing the leading '1. ', '2. ', etc.
        # Regex explanation:
        # ^      : Start of string
        # \d+    : One or more digits
        # \.     : A literal dot
        # \s*    : Optional whitespace following the dot
        category_name = re.sub(r'^\d+\.\s*', '', raw_name)
        
        color_key = color_row[0]
        
        if color_key:
            color_to_category[color_key] = category_name

    ################# Building the code mapping #################
    mapping_dict = {}
    
    glossary_texts = glossary_table['text'].rows()
    glossary_colors = glossary_table['colors'].rows()
    
    for text_row, color_row in zip(glossary_texts, glossary_colors):
        code = clean_text(text_row[0])
        description = clean_text(text_row[1])
        
        # Getting the color of the Code cell (column 0)
        row_color = color_row[0]
        
        # Looking up the category, default to 'General' if there's no color data
        category = color_to_category.get(row_color, 'General')
        
        if code:
            # Format: (Category) Description [Code]
            formatted_string = f'({category}) {description} [{code}]'
            
            # Final clean to ensure the concatenation didn't create double spaces
            mapping_dict[code] = clean_text(formatted_string)
            
    return mapping_dict

###################################################################

# Running the function to generate the renaming map and renaming the dataframe
renaming_map = create_dataset_mapping(svi2018_colors, svi2018_table)

df_clean['svi_2018'] = df_raw['svi_2018'].rename(renaming_map, strict=False)

print(df_clean['svi_2018'])

shape: (3_143, 125)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ (General) ┆ (General) ┆ (General) ┆ (General) ┆ … ┆ (General) ┆ (General) ┆ GeoLevel ┆ Compariso │
│ State-lev ┆ State     ┆ State abb ┆ County    ┆   ┆ Adjunct   ┆ Adjunct   ┆ ---      ┆ n         │
│ el FIPS   ┆ name      ┆ reviation ┆ name      ┆   ┆ variable  ┆ variable  ┆ str      ┆ ---       │
│ cod…      ┆ [STATE]   ┆ […        ┆ [COUNTY]  ┆   ┆ - P…      ┆ - E…      ┆          ┆ str       │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆          ┆           │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆          ┆           │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ ST        ┆ STATE     ┆ ST_ABBR   ┆ COUNTY    ┆ … ┆ MP_UNINSU ┆ E_DAYPOP  ┆          ┆           │
│           ┆           ┆           ┆           ┆   ┆ R         ┆      

In [144]:
df_clean['svi_2018'].columns

['(General) State-level FIPS code [ST]',
 '(General) State name [STATE]',
 '(General) State abbreviation [ST_ABBR]',
 '(General) County name [COUNTY]',
 '(General) Tract-level FIPS code [FIPS]',
 '(General) Text description of tract, county, state [LOCATION]',
 '(General) Tract area in square miles [AREA_SQMI]',
 '(General) Population estimate, 2014-2018 ACS [E_TOTPOP]',
 '(General) Population estimate MOE, 2014-2018 ACS [M_TOTPOP]',
 '(General) Housing units estimate, 2014-2018 ACS [E_HU]',
 '(General) Housing units estimate MOE, 2014-2018 ACS [M_HU]',
 '(General) Households estimate, 2014-2018 ACS [E_HH]',
 '(General) Households estimate MOE, 2014-2018 ACS [M_HH]',
 '(Socioeconomic) Persons below poverty estimate, 2014-2018 ACS [E_POV]',
 '(Socioeconomic) Persons below poverty estimate MOE, 2014-2018 ACS [M_POV]',
 '(Socioeconomic) Civilian (age 16+) unemployed estimate, 2014-2018 ACS [E_UNEMP]',
 '(Socioeconomic) Civilian (age 16+) unemployed estimate MOE, 2014-2018 ACS [M_UNEMP]',


In [145]:
# Slicing the first row, converting to float and dropping redundant columns
redundant_cols=['(General) State-level FIPS code [ST]',
                # '(General) State abbreviation [ST_ABBR]', we keep this column because some of the dataframes use state abbreviations instead of full names
                '(General) Text description of tract, county, state [LOCATION]',
                '(General) Tract area in square miles [AREA_SQMI]',
                #'(General) Tract-level FIPS code [FIPS]',
                'GeoLevel',
                'Comparison']

df_clean['svi_2018'] = df_clean['svi_2018'].drop(redundant_cols)

# The FIPS column may technically be an identifier column but we don't want it to accidentally
# be seen as a numerical feature during our analysis, so is not kept
text_cols = ['State', 'County', 'State Abbreviation']

# Manually renaming the state and county name columns to be directly
# compatible with the rest of our dataframes
df_clean['svi_2018'] = df_clean['svi_2018'].rename({'(General) State name [STATE]': 'State',
                                                    '(General) County name [COUNTY]': 'County',
                                                    '(General) State abbreviation [ST_ABBR]': 'State Abbreviation',
                                                    '(General) Tract-level FIPS code [FIPS]': 'FIPS'
                                                   })

# Dropping the margin of error columns
banned_terms = ['MOE']

clean_columns_svi = [
    c for c in df_clean['svi_2018'].columns 
    if not any(term in c for term in banned_terms)
]

df_clean['svi_2018'] = df_clean['svi_2018'].select(clean_columns_svi)

df_clean['svi_2018'] = convert_to_float(df_clean['svi_2018'].slice(1), text_cols)

# Final Cleanup

## County and State name standardization

In [146]:
# We have a lot of dataframes with different county naming conventions. In some, they have the word 'County' after the
# county name while in others they're all uppercase. This function attempts to unify the county names

def clean_geo_columns(df_dict):
    """
    Attempts to standardize county and state columns' grammar
    Does NOT remove 'County' or 'City' suffixes. 
    """
    cleaned_dict = {}

    # We need to define a special corrections map for Puerto Rico since the special characters in Puerto Rican
    # names break in regex, and also skip over Virginia counties because of the independent city counties
    # that share a name with existing counties, which will require special care
    pr_corrections = {
        'A?Asco': 'Añasco', 
        'Bayam?N': 'Bayamón', 
        'Can?Vanas': 'Canóvanas',
        'Cata?O': 'Cataño', 
        'Comer?O': 'Comerío', 
        'Gu?Nica': 'Guánica',
        'Juana D?Az': 'Juana Díaz', 
        'Las Mar?As': 'Las Marías', 
        'Lo?Za': 'Loíza',
        'Manat?': 'Manatí', 
        'Mayag?Ez': 'Mayagüez', 
        'Pe?Uelas': 'Peñuelas',
        'R?O Grande': 'Río Grande', 
        'Rinc?N': 'Rincón',
        'San Germ?N': 'San Germán', 
        'San Sebasti?N': 'San Sebastián'
    }

    for name, df in df_dict.items():
        cols = df.columns
        expressions = []

        # Cleaning State first (for reliable logic later)
        if 'State' in cols:
            df = df.with_columns(
                pl.col('State').cast(pl.Utf8).str.to_titlecase().str.strip_chars()
            )

        # Cleaning county (Casing & PR fixes only)
        if 'County' in cols:
            county_expr = (
                pl.col('County').cast(pl.Utf8)
                .str.to_titlecase()
                .str.strip_chars()
                .replace(pr_corrections, return_dtype=pl.Utf8)
            )
            expressions.append(county_expr.alias('County'))

        if expressions:
            cleaned_dict[name] = df.with_columns(expressions)
        else:
            cleaned_dict[name] = df

    return cleaned_dict


In [147]:
# The latitude/longitude dataframe only has state abbreviations, so we will need to create a new column. 
# Thankfully, the SVI dataframe covers all the states, so we can use the state name column from there

statenames_ref = (
    df_clean['svi_2018']
    .select(['State Abbreviation', 'State'])
    .unique()
)

df_clean['lat_long'] = df_clean['lat_long'].join(
    statenames_ref,
    on = 'State Abbreviation',
    how = 'left'
)

df_clean['lat_long'] = df_clean['lat_long'].with_columns(
    pl.col('State').str.to_titlecase()
)


# The SVI dataframe is the most comprehensive one in terms of state names, but some territories still get glossed over
# by this join due to them not being represented in SVI but represented in other dataframes. We can manually handle these
print(
    df_clean['lat_long']
    .filter(pl.col('State').is_null())
    .select('State Abbreviation')
    .unique()
)

df_clean['lat_long'] = df_clean['lat_long'].with_columns(
    pl.col('State').fill_null(
        pl.col('State Abbreviation').replace({
            'PR': 'Puerto Rico'
        })
    )
)

print(df_clean['lat_long'])

shape: (1, 1)
┌────────────────────┐
│ State Abbreviation │
│ ---                │
│ str                │
╞════════════════════╡
│ PR                 │
└────────────────────┘
shape: (3_221, 7)
┌────────────────┬────────────────┬────────────┬────────────┬───────────┬────────────┬─────────────┐
│ State          ┆ County         ┆ Land Area  ┆ Water Area ┆ Latitude  ┆ Longitude  ┆ State       │
│ Abbreviation   ┆ ---            ┆ ---        ┆ ---        ┆ ---       ┆ ---        ┆ ---         │
│ ---            ┆ str            ┆ i64        ┆ i64        ┆ f64       ┆ str        ┆ str         │
│ str            ┆                ┆            ┆            ┆           ┆            ┆             │
╞════════════════╪════════════════╪════════════╪════════════╪═══════════╪════════════╪═════════════╡
│ AL             ┆ Autauga County ┆ 1539634184 ┆ 25674812   ┆ 32.532237 ┆ -86.64644  ┆ Alabama     │
│ AL             ┆ Baldwin County ┆ 4117656514 ┆ 1132955729 ┆ 30.659218 ┆ -87.746067 ┆ Alabama     │

In [148]:
# Unfortunately, we also have to ensure the county naming convention is consistent across the datasets,
# which is a very cumbersome process considering all the edge cases like city states and format
# differences.

def standardize_county_names(df, name_col = 'County', state_col = 'State'):
    
    ########################## County Naming Case Configuration ##########################
    
    # Case 1: The counties that aren't city counties but have the word 'City' in their name regardless
    city_named_counties = ['James City', 'Charles City']

    # Case 2: City counties that don't have the word 'City' officially in their name. 
    # These may or may NOT have the word 'City' appended to their name based on dataset
    independent_cities = [
        'Alexandria', 'Bristol', 'Buena Vista', 'Charlottesville', 'Chesapeake',
        'Colonial Heights', 'Covington', 'Danville', 'Emporia', 'Falls Church',
        'Fredericksburg', 'Galax', 'Hampton', 'Harrisonburg', 'Hopewell',
        'Lexington', 'Lynchburg', 'Manassas', 'Manassas Park', 'Martinsville',
        'Newport News', 'Norfolk', 'Norton', 'Petersburg', 'Poquoson',
        'Portsmouth', 'Radford', 'Salem', 'Staunton', 'Suffolk',
        'Virginia Beach', 'Waynesboro', 'Williamsburg', 'Winchester', 'Carson'
    ]

    # Case 3: The more complex cases, like non-English names that may or may not have spaces or abbreviated words in their names
    # plus the Alaskan regions that have had their classification convention changed around the time window these datasets cover.
    overrides = {
        # Alaska Standardizations (Forcing them to match the 2020 Gazetteer exact naming)
        'Chugach County': 'Chugach Census Area',
        'Copper River County': 'Copper River Census Area',
        'Kusilvak County': 'Kusilvak Census Area',
        'Juneau County': 'Juneau City and Borough',
        'Sitka County': 'Sitka City and Borough',
        'Wrangell County': 'Wrangell City and Borough',
        'Yakutat County': 'Yakutat City and Borough',
        'Anchorage County': 'Anchorage Municipality',
        
        # Typos / Duplicates
        'Lasalle County': 'La Salle County',
        'Dona Ana County': 'Doña Ana County',
        'De Kalb County': 'DeKalb County',
        'Fond Du Lac County': 'Fond du Lac County',
        'Saint Lucie County': 'St. Lucie County',
    }

    ########################## Setting the Regex ##########################
    # This regex pattern matches 'City and County', 'City and Burrough', 'Census Area', etc.
    # We DO NOT include just 'City' here, to protect 'Richmond City'.
    suffix_pattern = r'(?i)\s+(City\s+and\s+(Borough|County|Burrough)|Planning\s+Region|Census\s+Area|Municipality|Municipio|Borough|Burrough|Parish|County)$'

    return df.with_columns(
        pl.col(name_col)
        ######## Cleanup
        .str.strip_chars()
        .str.to_titlecase()
        .str.replace(r'(?i)\bSt\.\s', 'Saint ')
        .str.replace(r'(?i)\bSt\s', 'Saint ')
        .str.replace(r'(?i)\bSte\.\s', 'Sainte ')
        
        ######## Greedy Strip
        # 'Juneau City and County' -> 'Juneau'
        # 'Wrangell City and Burrough' -> 'Wrangell'
        .str.replace(suffix_pattern, '')
        .alias('_root')
    ).with_columns(

        ######## Rebuild Logic
        pl.when(pl.col('_root').is_in(city_named_counties))
        .then(pl.col('_root') + ' County')
        
        .when(pl.col('_root').str.replace(r'(?i)\s+City$', '').is_in(independent_cities))
        .then(pl.col('_root').str.replace(r'(?i)\s+City$', '') + ' City')
        
        .when(pl.col('_root').str.contains(r'(?i) City$'))
        .then(pl.col('_root'))
        
        .otherwise(pl.col('_root') + ' County')
        .alias(name_col)
    ).with_columns(

        ######## Apply Overrides
        pl.col(name_col).replace(overrides)
    ).with_columns(

        ######## Wisconsin Fix (Revert Juneau AK -> Juneau WI if state matches)
        pl.when(
            (pl.col(state_col) == 'Wisconsin') & 
            (pl.col(name_col) == 'Juneau City and Borough')
        )
        .then(pl.lit('Juneau County'))
        .otherwise(pl.col(name_col))
        .alias(name_col)
    ).drop('_root')

####################################################################

# Running the functions

# Popping the state-level data to avoid county-level function malfunctions
df_resp = df_clean.pop('resp_therapist_per_state')
df_md = df_clean.pop('md_per_state')


df_clean2 = clean_geo_columns(df_clean)

df_unified = {}

for name, df in df_clean2.items():
    df_unified[name] = standardize_county_names(df)

# df_clean['resp_therapist_per_state'] = df_resp
# df_clean['md_per_state'] = df_md

C:\Users\mpola\AppData\Local\Temp\ipykernel_21064\2095849978.py:49: DeprecationWarning: the `return_dtype` parameter for `replace` is deprecated. Use `replace_strict` instead to set a return data type while replacing values.
(Deprecated in version 1.0.0)
  .replace(pr_corrections, return_dtype=pl.Utf8)


## Creating the full counties list

In [149]:
# While we can use FIPS codes as index, they're not as easily comprehensible as county and state names
# However, county names are not unique across states, but unique within states. A combined 'County (State)'
# column is both readable and unique.

def generate_geo_keys(df_dict):
    """
    Generates a composite 'County (State)' key and reorders columns.
    """
    final_dict = {}
    
    for name, df in df_dict.items():
        cols = df.columns
        
        # We only create the key if BOTH columns exist
        if 'County' in cols and 'State' in cols:
            new_col_name = 'County (State)'
            
            # Creating the composite key using the existing columns
            df = df.with_columns(
                pl.format('{} ({})', pl.col('County'), pl.col('State')).alias(new_col_name)
            )
            
            # Reorder: County (State), County, State, [rest of the features]
            df = df.select(
                new_col_name,
                'County',
                'State',
                pl.exclude([new_col_name, 'County', 'State'])
            )
            
        final_dict[name] = df
        
    return final_dict

###########################################

df_keyed = generate_geo_keys(df_unified)

In [150]:
# When running our analysis we will start with imputing missing data for all counties, so we 
# need a reliable dataframe to join everything else onto that has rows for all counties

def create_master_county_list(dfs_dict, key_cols=['County (State)', 'County', 'State']):
    """
    Creates a reference dataframe of unique counties from a dictionary of DataFrames.
    """
    
    # Extracting the key columns from each dataframe in the dictionary
    key_frames = (df.select(key_cols) for df in dfs_dict.values())
    
    # Vertically concatenating all these frames, keeping unique rows
    reference_df = (
        pl.concat(key_frames)
        .unique(subset=key_cols)
    )
    
    return reference_df

######################################################

# We define a dataframe outside of the dictionary first, and then add it to the dictionary

counties_list = create_master_county_list(df_keyed).sort(['State', 'County'])

print(counties_list)

shape: (3_225, 3)
┌─────────────────────────────┬───────────────────┬─────────┐
│ County (State)              ┆ County            ┆ State   │
│ ---                         ┆ ---               ┆ ---     │
│ str                         ┆ str               ┆ str     │
╞═════════════════════════════╪═══════════════════╪═════════╡
│ Autauga County (Alabama)    ┆ Autauga County    ┆ Alabama │
│ Baldwin County (Alabama)    ┆ Baldwin County    ┆ Alabama │
│ Barbour County (Alabama)    ┆ Barbour County    ┆ Alabama │
│ Bibb County (Alabama)       ┆ Bibb County       ┆ Alabama │
│ Blount County (Alabama)     ┆ Blount County     ┆ Alabama │
│ …                           ┆ …                 ┆ …       │
│ Sweetwater County (Wyoming) ┆ Sweetwater County ┆ Wyoming │
│ Teton County (Wyoming)      ┆ Teton County      ┆ Wyoming │
│ Uinta County (Wyoming)      ┆ Uinta County      ┆ Wyoming │
│ Washakie County (Wyoming)   ┆ Washakie County   ┆ Wyoming │
│ Weston County (Wyoming)     ┆ Weston County     ┆ 

In [151]:
# We also want the reference dataframe to have FIPS data, to ensure we get no issues when
# adding border data later on. We can use the existing dataframes' FIPS data for now

def clean_fips_expr(col_name):
    """
    Standardizes FIPS to 5-digit string with leading zeros.
    - Handles floats: 1001.0 -> '01001'
    - Handles ints: 1001 -> '01001'
    - Handles strings: '1001' -> '01001'
    """
    return (
        pl.col(col_name)
        # Casting to Float64 first to handle strings like '1001.0' without error
        .cast(pl.Float64, strict=False)

        # Casting to Int64 to remove the decimal point (1001.0 -> 1001)
        .cast(pl.Int64, strict=False)

        # Casting to String to allow padding
        .cast(pl.String)

        # Padding start with '0' to ensure 5 digits (e.g. '1001' -> '01001')
        .str.pad_start(5, '0') 
    )

######################################################

def merge_fips(target_df, 
               source_df, 
               join_key = 'County (State)', 
               source_fips_col = 'FIPS'):
    # Cleaning Source FIPS
    source_clean = source_df.select([
        pl.col(join_key),
        clean_fips_expr(source_fips_col).alias('FIPS_new')
    ])

    # Checking if Target already has FIPS
    if 'FIPS' in target_df.columns:
        # Cleaning existing Target FIPS so we compare apples-to-apples
        # This ensures '1001' in target matches '01001' in source
        target_df = target_df.with_columns(clean_fips_expr('FIPS').alias('FIPS'))
        
        # Performing the join
        joined = target_df.join(source_clean, on=join_key, how='left')
        
        # Detecting conflicts (Where both exist but differ)
        conflicts = joined.filter(
            pl.col('FIPS').is_not_null() & 
            pl.col('FIPS_new').is_not_null() & 
            (pl.col('FIPS') != pl.col('FIPS_new'))
        )
        
        if not conflicts.is_empty():
            print(f'=== WARNING: Found {conflicts.height} FIPS mismatches! ===')
            print(conflicts.select([join_key, 'FIPS', 'FIPS_new']).head())
        else:
            print('=== No conflicts found. FIPS codes matched or filled successfully. ===')
            
        # Coalescing: Keep existing FIPS if present, otherwise use new, then drop temp column
        return joined.with_columns(
            pl.coalesce(['FIPS', 'FIPS_new']).alias('FIPS')
        ).drop('FIPS_new')

    else:
        print('=== Target has no FIPS column. Creating new standardized one. ===')
        return (
            target_df
            .join(source_clean, on=join_key, how='left')
            .rename({'FIPS_new': 'FIPS'})
        )
    
######################################

df_keyed['5yr_private_insurance'] = df_keyed['5yr_private_insurance'].with_columns(
    pl.col('Geography').str.slice(-5).alias('FIPS')
)

counties_list2 = merge_fips(counties_list, df_keyed['svi_2018'])
counties_list2 = merge_fips(counties_list2, df_keyed['county_additional_measures'])
counties_list2 = merge_fips(counties_list2, df_keyed['5yr_private_insurance'])
counties_list2 = merge_fips(counties_list2, df_keyed['pollution'])
counties_list2 = merge_fips(counties_list2, df_keyed['asthma'])

counties_list2['FIPS'].unique()

=== Target has no FIPS column. Creating new standardized one. ===
=== No conflicts found. FIPS codes matched or filled successfully. ===
=== No conflicts found. FIPS codes matched or filled successfully. ===
=== No conflicts found. FIPS codes matched or filled successfully. ===
=== No conflicts found. FIPS codes matched or filled successfully. ===


FIPS
str
"""37097"""
"""40067"""
"""51520"""
"""47159"""
"""37025"""
…
"""48447"""
"""55057"""
"""55061"""


In [152]:
# Adding the previously popped dataframes back onto the dictionary
df_keyed['counties'] = counties_list2
df_keyed['resp_therapist_per_state'] = df_resp
df_keyed['md_per_state'] = df_md

In [153]:
# Troubleshooting and quality control
# This function ensures that we have exactly one row per county, and that we did not accidentally
# misname any county in any dataframe and can join them later on without issue.
# We will use the counties table as the base table since it includes all counties for all states.

def audit_join_keys(df_dict, state_keys, join_key_col='County (State)'):
    """
    Checks every dataframe to ensure it is unique on the intended join key.
    """
    print(f"{'DataFrame Name':<30} | {'Key Used':<15} | {'Unique?':<8} | {'Duplicates'}")
    print('-' * 80)

    for name, df in df_dict.items():
        # Handle LazyFrames if necessary
        if isinstance(df, pl.LazyFrame):
            df = df.collect()
            
        # Determining which key we expect to be unique
        if name in state_keys:
            key = 'State'
        else:
            key = join_key_col
            
        # Checking for duplicates on that key
        is_unique = df.n_unique(subset=[key]) == len(df)
        
        # Calculating how many extra rows exist
        total_rows = len(df)
        unique_keys = df.n_unique(subset=[key])
        duplicates = total_rows - unique_keys
        
        status = 'PASS' if is_unique else 'FAIL'
        
        print(f"{name:<30} | {key:<15} | {status:<8} | {duplicates}")
        
        # If it failed, print a sample of the offenders
        if not is_unique:
            print(f'   >>> EXAMPLE DUPLICATES in {name}:')
            dupe_examples = (
                df.filter(pl.len().over(key) > 1)
                .sort(key)
                .select([key] + [c for c in df.columns if c != key][:2])
            )
            print(dupe_examples.unique())
            print('-' * 80)

#######################################################

state_level_keys = ['resp_therapist_per_state', 'md_per_state']

audit_join_keys(df_keyed, state_level_keys)


DataFrame Name                 | Key Used        | Unique?  | Duplicates
--------------------------------------------------------------------------------
lat_long                       | County (State)  | PASS     | 0
5yr_demographic                | County (State)  | PASS     | 0
5yr_private_insurance          | County (State)  | PASS     | 0
5yr_public_insurance           | County (State)  | PASS     | 0
pollution                      | County (State)  | PASS     | 0
asthma                         | County (State)  | PASS     | 0
county_health_measures         | County (State)  | PASS     | 0
county_additional_measures     | County (State)  | PASS     | 0
svi_2018                       | County (State)  | PASS     | 0
counties                       | County (State)  | PASS     | 0
resp_therapist_per_state       | State           | PASS     | 0
md_per_state                   | State           | PASS     | 0


In [154]:
# Manually checking the county counts per state to ensure we're not deviating from the expected counts

def print_formatted_county_audit(df, counties_per_line = 8, print_list = True):
    """
    Prints a detailed audit of counties per state, formatting the list
    into multiple lines to avoid truncation.
    """
    # Grouping by State, counting rows, and collecting sorted list of counties
    summary = (
        df.group_by('State')
        .agg([
            pl.len().alias('Count'),
            pl.col('County').sort().alias('Counties') # Sorting alphabetically for easier comparison to official records
        ])
        .sort('State')
    )
    state_counter = 1
    county_counter = 0

    # Iterating through every state
    for row in summary.iter_rows(named=True):
        state = row['State']
        count = row['Count']
        county_list = row['Counties']
        
        # Printing the header
        print(f'({state_counter}.) State: {state} | Total: {count}')
        print('-' * 50)
        state_counter += 1

        # Chunking the list and print N items per line
        for i in range(0, len(county_list), counties_per_line):
            # Slicing the list to get the next batch
            chunk = county_list[i : i + counties_per_line]
            county_counter += len(chunk)
            
            # Join them with a comma and print
            if print_list:
                print(', '.join(chunk))
            
        print('\n') # Add a blank line between states for separation
    print(f'Total county count: {county_counter}')

###############################################

print_formatted_county_audit(counties_list, 
                             counties_per_line=16, 
                             print_list=True)

(1.) State: Alabama | Total: 67
--------------------------------------------------
Autauga County, Baldwin County, Barbour County, Bibb County, Blount County, Bullock County, Butler County, Calhoun County, Chambers County, Cherokee County, Chilton County, Choctaw County, Clarke County, Clay County, Cleburne County, Coffee County
Colbert County, Conecuh County, Coosa County, Covington City, Crenshaw County, Cullman County, Dale County, Dallas County, Dekalb County, Elmore County, Escambia County, Etowah County, Fayette County, Franklin County, Geneva County, Greene County
Hale County, Henry County, Houston County, Jackson County, Jefferson County, Lamar County, Lauderdale County, Lawrence County, Lee County, Limestone County, Lowndes County, Macon County, Madison County, Marengo County, Marion County, Marshall County
Mobile County, Monroe County, Montgomery County, Morgan County, Perry County, Pickens County, Pike County, Randolph County, Russell County, Saint Clair County, Shelby Count

# Singular Dataframe Construction

## Dataframe Merging

In [155]:
# Merging all dataframes into one and dropping unneeded string columns
# We start off by ensuring that the FIPS columns are all set to string so that the merge can happen with minimal issue
for name in df_keyed:
    if 'FIPS' in df_keyed[name].columns:
        df_keyed[name] = df_keyed[name].with_columns(
            pl.col('FIPS')
            .cast(pl.String)              # 1. Cast to String
            .str.split('.').list.get(0)   # 2. Get part before '.' (Corrected)
            .str.pad_start(5, '0')        # 3. Add leading zeros
            .alias('FIPS')                # 4. Keep the same name
        )
        print(f'=== Standardized FIPS in {name} ===')

# While we will join by the County (State) column, the respiratory therapist
# and MD datasets are still recorded on a per-state basis and will require an exception
# to be made
state_level_keys = ['resp_therapist_per_state', 'md_per_state']


# We will use the counties table as the base table since it covers all counties present in other dataframes
df_base = df_keyed.pop('counties').lazy()


for name, df_next in df_keyed.items():
    df_lazy = df_next.lazy()
    
    if name in state_level_keys:
        # State-Level Join
        # Joining on State Code only. 
        # This broadcasts state info to every county in that state.
        df_base = df_base.join(
            df_lazy,
            on='State',
            how='left',
            suffix=f'_{name}'
        )
    else:
        # County-Level Join
        # Joining on new composite key 'County (State)'
        # This prevents 'Orange (California)' from matching with 'Orange (Florida)'
        df_clean = df_lazy.select(pl.exclude(['State', 'County']))

        df_base = df_base.join(
            df_clean,
            on='County (State)', 
            how='left',
            suffix=f'_{name}'
        )

# Finalizing
df_base = df_base.collect()

=== Standardized FIPS in 5yr_private_insurance ===
=== Standardized FIPS in pollution ===
=== Standardized FIPS in asthma ===
=== Standardized FIPS in county_health_measures ===
=== Standardized FIPS in county_additional_measures ===
=== Standardized FIPS in svi_2018 ===
=== Standardized FIPS in counties ===


In [156]:
# Because of our initial loading step misrecognizing the headers as a regular row of data,
# All the features are labeled as strings in some of the datasets by polars. This needs to be remedied.

def safe_convert_numeric_strings(df, exclude_list = []):
    # Identifying all string columns
    string_cols = df.select(cs.string()).columns
    
    convert_exprs = []
    
    for col in string_cols:
        if col not in exclude_list:
            # Defining the conversion logic:
            # - Remove commas (common in CSVs: '1,000')
            # - Cast to Float64 with strict=False (turns failures into nulls instead of crashing)
            conversion = pl.col(col).str.replace_all(',', '').cast(pl.Float64, strict=False)
            
            # 'Safety Check': Should we actually apply this?
            # We check if the conversion results in valid numbers.
            # If a column is mostly text (like 'Alabama'), casting makes it all null.
            # We check if the count of valid items > 0.
            
            # Fast check: compare null counts.
            # (We do this eagerly to decide whether to add the expression)
            is_numeric = df.select(
                conversion.is_not_null().sum() > 0
            ).item()
            
            if is_numeric:
                convert_exprs.append(conversion)

    # Applying all valid conversions at once
    if convert_exprs:
        return df.with_columns(convert_exprs)
    return df

###########################################

df_numeric = safe_convert_numeric_strings(df_base, exclude_list = ['FIPS'])

In [157]:
# Ensuring that the string columns that are not part of our key identifiers are dropped

keep_list = ['County (State)', 'State', 'State Abbreviation', 'FIPS']
asthma_columns = ['Age-adjusted ER Visit Rate for Asthma per 10,000 People',
                  'Age-adjusted Hospitalization Rate for Asthma per 10,000 People']


string_cols = [
    col for col, dtype in df_numeric.schema.items() 
    if dtype in (pl.Utf8, pl.String)
]

print(f'=== Found {len(string_cols)} string columns in total: ===')
for i in string_cols:
    if i not in keep_list:
        print(i)
print('=============================================')

# Determining which string columns are NOT in the keep list
drop_list = [col for col in string_cols if col not in keep_list]

df_dropped = df_numeric.drop(drop_list)

# Dropping the columns missing from at least 40% of rows. We will drop columns
# that are missing from counties with asthma-related data at a later step, but these
# features are mostly just noise. We keep the threshold relatively low for that reason:
# A feature may be missing on all counties without asthma data but present in those with it,
# which would result in a 100% reliable feature for our analysis but get dropped because it is
# missing from at least 50% of the counties.

total_rows = df_dropped.height
null_stats = df_dropped.null_count().row(0, named=True)

THRESHOLD = 0.40

cols_to_drop = []
for col_name, null_count in null_stats.items():
    # Checking if nulls are below threshold and ensuring the 
    # column is not in our exclusion list (other targets)

    if (null_count / total_rows) > THRESHOLD and col_name not in asthma_columns:
        cols_to_drop.append(col_name)
        print(f'Dropping column {col_name} due to {null_count} nulls ({(null_count / total_rows) * 100:.2f}%)')

print(f'Total columns to drop: {len(cols_to_drop)}')

df_dropped = df_dropped.drop(cols_to_drop)

=== Found 27 string columns in total: ===
County
Geography
Geography_5yr_private_insurance
Geography_5yr_public_insurance
Data Comment_Pollutant: Benzene
Data Comment_Pollutant: Formaldehyde
Data Comment_Pollutant: Acetaldehyde
Data Comment_Pollutant: Carbon tetrachloride
Data Comment_Pollutant: 1,3-butadiene
Data Comment_Pollutant: Napthalene
Data Comment_Pollutant: Ethylene oxide
Data Comment_Pollutant: Diesel Particulate Matter
ER Visit Comment
Hospitalization Comment
Low birthweight => Unreliable
Primary care physicians => PCP Ratio
Dentists => Dentist Ratio
Mental health providers => MHP Ratio
Drinking water violations => Presence of violation
Other primary care providers => Other PCP Ratio
State Abbreviation_svi_2018
Source
Source_md_per_state
Dropping column Geographic Area Name due to 3225 nulls (100.00%)
Dropping column Percent => Total population due to 3225 nulls (100.00%)
Dropping column Percent => Total population => SUMMARY INDICATORS => Median age (years) due to 3225 nul

## Turning Aggregates to Rates

In [158]:
# We have multiple population tracking columns in the merged dataframe. A lot of them are redundant,
# so we will be picking one to be our 'basis', and this function's goal is to confirm that
# the ACS 'Total Population' aggregate is the most comprehensive one and, if there exist counties
# that have population data in another population feature but not in this one, append them onto
# this one.

def analyze_coverage_fast(df, ref_col, target_cols):
    expressions = []
    
    for col in target_cols:
        # Match Count: Where values are identical
        expressions.append(
            (pl.col(ref_col) == pl.col(col)).sum().alias(f'{col}_Match Count')
        )
        # Extras Count: Where reference is null but candidate has data
        expressions.append(
            (pl.col(ref_col).is_null() & pl.col(col).is_not_null()).sum().alias(f'{col}_Extras Count')
        )
        
    # Process the results into a clean table
    summary = (
        df.select(expressions)
        .unpivot()
        .with_columns([
            # Split the variable into the original column name and the metric name
            pl.col('variable').str.extract(r'^(.*)_(Match Count|Extras Count)$', 1).alias('Feature Name'),
            pl.col('variable').str.extract(r'^(.*)_(Match Count|Extras Count)$', 2).alias('metric')
        ])
        .pivot(index='Feature Name', on='metric', values='value')
        .select([
            pl.col('Feature Name'),
            pl.col('Match Count').cast(pl.Int64),
            pl.col('Extras Count').cast(pl.Int64)
        ])
        .sort('Extras Count', descending=True)
    )
    
    return summary

###############################################
pop_cols = {
    col: df_dropped[col].null_count() 
    for col in df_dropped.columns if 'population' in col.lower()
}

analyze_coverage_fast(df_dropped, 'Total => Total population', pop_cols)

Feature Name,Match Count,Extras Count
str,i64,i64
"""Population, All (State Level F…",0,2
"""Population, All (County Level …",0,2
"""M.D per 100,000 population""",0,2
"""Total => Total population""",3220,0
"""Total => Total population => S…",0,0
…,…,…
"""(Household Composition/Disabil…",0,0
"""(General) Adjunct variable - U…",0,0
"""(General) Adjunct variable - P…",0,0


In [159]:
# We have some aggregate features that we'd prefer if they were ratios, since populations and population
# densities vary wildly between counties. This function 'tests' numeric columns to see if they're
# appropriate to convert to percentage and does so using existing population data.

def convert_aggregates_to_rates(
    df, 
    pop_col, 
    exclude_cols = [], 
    exclude_keywords = [],
    correlation_threshold = 0.85
):
    """
    Converts aggregate columns to rates per 100k using Polars.
    
    Args:
        df: Input Polars DataFrame
        pop_col: Name of the population column (denominator)
        exclude_cols: Exact list of columns to skip (e.g., 'FIPS', 'state_fips')
        exclude_keywords: List of substrings to skip (e.g., ['median', 'avg', 'rate', 'index'])
        correlation_threshold: Pearson correlation cutoff to classify a feature as an aggregate
    
    Returns:
        pl.DataFrame with converted rates and original aggregates dropped.
    """

    # Always exclude the denominator and coordinates/IDs
    exclude_cols.extend([pop_col, 'Latitude', 'Longitude', 'geometry'])
    
    numeric_cols = [
        col for col, dtype in df.schema.items() 
        if dtype in (pl.Float64, pl.Float32, pl.Int64, pl.Int32)
        and col not in exclude_cols
        and not any(kw in col.lower() for kw in exclude_keywords)
    ]

    excluded_cols = [col for col, dtype in df.schema.items()
                     if any(kw in col.lower() for kw in exclude_keywords)]

    print(f'Excluded {len(excluded_cols)} columns due to keywords.')
    print(f'Scanning {len(numeric_cols)} columns for aggregate counts')

    # Calculating stats for all columns (we can do this in a single pass with Polars)
    # We get Min, Max, and Correlation to Population for every candidate column.
    stats_exprs = []
    for c in numeric_cols:
        stats_exprs.extend([
            pl.col(c).min().alias(f'{c}_min'),
            pl.col(c).max().alias(f'{c}_max'),
            pl.corr(c, pop_col).alias(f'{c}_corr')
        ])
    
    # Running the query to get a single row of stats
    stats_row = df.select(stats_exprs).to_dicts()[0]

    cols_to_convert = []
    skipped_as_percentage = []

    # Evaluation Logic
    for col in numeric_cols:
        c_min  = stats_row[f'{col}_min']
        c_max  = stats_row[f'{col}_max']
        c_corr = stats_row[f'{col}_corr']
        
        # Handling all-null columns
        if c_min is None or c_max is None:
            continue

        # RULE A: The 'Percentage/Index' Check
        # If data is strictly between 0 and 100, assume it is already a rate/percentage
        if c_min >= 0 and c_max <= 100:
            skipped_as_percentage.append(col)
            continue
            
        # RULE B: The 'Aggregate' Check
        # If it correlates highly with population, it's a count.
        if c_corr is not None and abs(c_corr) > correlation_threshold:
            cols_to_convert.append(col)

    print(f'=== Skipped {len(skipped_as_percentage)} columns automatically (Bounded 0-100) ===')
    for i in skipped_as_percentage:
        print(i)
    print('=============================================')

    print(f'=== Identified {len(cols_to_convert)} aggregate columns to convert: ===')
    for i in cols_to_convert:
        print(i)
    print('=============================================')

    # Applying the  conversions
    expressions = [
        (pl.col(c) / pl.col(pop_col) * 100_000).alias(f'{c} (per 100k)') 
        for c in cols_to_convert
    ]

    if expressions:
        return df.with_columns(expressions).drop(cols_to_convert)
    return df

###################################################

df_final = convert_aggregates_to_rates(df_dropped, 
                                       pop_col='Total => Total population', # The most comprehensive population column
                                       exclude_cols = [],
                                       exclude_keywords = ['median', 'average', 'mean', 
                                                           'rate', 'ratio', 'index', 'percent', 
                                                           'mode', 'per capita', '%', 
                                                           'value_pollutant:', 'per 10',
                                                           'life expectancy', 'mortality', 'days',
                                                           'severe housing cost burden', 'spl', 'sum of flags', 
                                                           'ep_', 'area', 'cohort size'],
                                       correlation_threshold = 0.8
                                                           )

Excluded 311 columns due to keywords.
Scanning 46 columns for aggregate counts
=== Skipped 2 columns automatically (Bounded 0-100) ===
Severe housing problems => Overcrowding
Severe housing problems => Inadequate Facilities
=== Identified 33 aggregate columns to convert: ===
Alcohol-impaired driving deaths => # Alcohol-Impaired Driving Deaths
Alcohol-impaired driving deaths => # Driving Deaths
Sexually transmitted infections => # Chlamydia Cases
Primary care physicians => # Primary Care Physicians
Dentists => # Dentists
Mental health providers => # Mental Health Providers
Some college => Population
Unemployment => Labor Force
Children in single-parent households => # Households
Social associations => # Associations
Injury deaths => # Injury Deaths
Long commute - driving alone => # Workers who Drive Alone
HIV prevalence => # HIV Cases
Motor vehicle crash deaths => # Motor Vehicle Deaths
Firearm fatalities => # Firearm Fatalities
Demographics => Population
(General) Population estimate, 

In [160]:
# Getting rid of the duplicate population columns
population_cols = [col for col in df_final.columns if 'population' in col.lower()]

for i in population_cols:
    if ('percent' not in i.lower()) and ('(per 100k)' not in i.lower()):
        print(f'"{i}",')

"Total => Total population",
"Total => Total population => SUMMARY INDICATORS => Median age (years)",
"Total => Total population => SUMMARY INDICATORS => Sex ratio (males per 100 females)",
"Total => Total population => SUMMARY INDICATORS => Age dependency ratio",
"Total => Total population => SUMMARY INDICATORS => Old-age dependency ratio",
"Total => Total population => SUMMARY INDICATORS => Child dependency ratio",
"Population, All (State Level File)",
"Respiratory Therapist per 100,000 population",
"Population, All (County Level File)",
"M.D per 100,000 population",


In [161]:
# Getting rid of the duplicate population columns

redundant_population_cols = [
    #"Total => Total population",
    #"Total => Total population => SUMMARY INDICATORS => Median age (years)",
    #"Total => Total population => SUMMARY INDICATORS => Sex ratio (males per 100 females)",
    #"Total => Total population => SUMMARY INDICATORS => Age dependency ratio",
    #"Total => Total population => SUMMARY INDICATORS => Old-age dependency ratio",
    #"Total => Total population => SUMMARY INDICATORS => Child dependency ratio",
    "Population, All (State Level File)",
    #"Respiratory Therapist per 100,000 population",
    "Population, All (County Level File)",
    #"M.D per 100,000 population",
    ]

df_final = df_final.drop(redundant_population_cols)

In [162]:
# Checking the other columns that were recognized as aggregate statistic

per_100k_cols = [col for col in df_final.columns if ' (per 100k)'.lower() in col.lower()]

for i in per_100k_cols:
    print(f'"{i}",')

"Alcohol-impaired driving deaths => # Alcohol-Impaired Driving Deaths (per 100k)",
"Alcohol-impaired driving deaths => # Driving Deaths (per 100k)",
"Sexually transmitted infections => # Chlamydia Cases (per 100k)",
"Primary care physicians => # Primary Care Physicians (per 100k)",
"Dentists => # Dentists (per 100k)",
"Mental health providers => # Mental Health Providers (per 100k)",
"Some college => Population (per 100k)",
"Unemployment => Labor Force (per 100k)",
"Children in single-parent households => # Households (per 100k)",
"Social associations => # Associations (per 100k)",
"Injury deaths => # Injury Deaths (per 100k)",
"Long commute - driving alone => # Workers who Drive Alone (per 100k)",
"HIV prevalence => # HIV Cases (per 100k)",
"Motor vehicle crash deaths => # Motor Vehicle Deaths (per 100k)",
"Firearm fatalities => # Firearm Fatalities (per 100k)",
"Demographics => Population (per 100k)",
"(General) Population estimate, 2014-2018 ACS [E_TOTPOP] (per 100k)",
"(General) Ho

## FIPS Uniqueness Testing

In [163]:
# Now we need to make sure that our naming scheme did not create any issues in terms of
# county uniqueness and coverage. 

fips_duplicates = df_final.filter(pl.col('FIPS').is_duplicated())

print('=== Counties with missing/non-unique FIPS codes: ===')
for row in fips_duplicates.rows(named=True):
    print(f"{row['County (State)']} : {row['FIPS']}, ")

=== Counties with missing/non-unique FIPS codes: ===
Chugach Census Area (Alaska) : None, 
Copper River Census Area (Alaska) : None, 


In [164]:
# With only two counties missing FIPS values, its trivial to manually look up and assign these values.

df_final = df_final.with_columns(
    pl.when(pl.col('County (State)') == 'Chugach Census Area (Alaska)')
    .then(pl.lit('02063'))
    .when(pl.col('County (State)') == 'Copper River Census Area (Alaska)')
    .then(pl.lit('02066'))
    .otherwise(pl.col('FIPS')) # Keep all other existing FIPS
    .alias('FIPS')
)

fips_duplicates = df_final.filter(pl.col('FIPS').is_duplicated())

print('=== Counties with missing/non-unique FIPS codes: ===')
for row in fips_duplicates.rows(named=True):
    print(f"{row['County (State)']} : {row['FIPS']}, ")

=== Counties with missing/non-unique FIPS codes: ===


In [165]:
# Dropping the redundant FIPS columns that may have bypassed our previous checks
fips_cols = [col for col in df_final.columns if 'FIPS_' in col]
for i in fips_cols:
    print(i)

df_final = df_final.drop([col for col in df_final.columns if 'FIPS_' in col])

FIPS_5yr_private_insurance
FIPS_pollution
FIPS_county_health_measures
FIPS_county_additional_measures
FIPS_svi_2018


# Writing the cleaned dataframes

In [166]:
# At this point we can write the new dataframes into disk as is, and load them later to work with

output_dir = r'C:\Users\mpola\asthma analysis\files\cleaned files'

# Looping through the dictionary and writing the CSVs
for name, df in df_keyed.items():
    # Constructing the full file path: folder + key + .csv
    file_path = os.path.join(output_dir, f'{name}.csv')

    # Writing the dataframe to CSV
    df.write_csv(file_path)
    print(f'Saved {file_path}')

df_final.write_csv(os.path.join(output_dir, 'merged_asthma_data.csv'))

Saved C:\Users\mpola\asthma analysis\files\cleaned files\lat_long.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\5yr_demographic.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\5yr_private_insurance.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\5yr_public_insurance.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\pollution.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\asthma.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\county_health_measures.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\county_additional_measures.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\svi_2018.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\resp_therapist_per_state.csv
Saved C:\Users\mpola\asthma analysis\files\cleaned files\md_per_state.csv
